# Setup
(No need to read)

In [ ]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-kihdyo3v
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-kihdyo3v
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit fa287750606075574df2c538058e67d648e2f952
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.7 MB/s eta 0:00:00
    

In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [ ]:
# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [ ]:
model = HookedTransformer.from_pretrained("gpt2-small")

Loaded pretrained model gpt2-small into HookedTransformer


# Numeric, decr

In [ ]:
example_prompt = "5 4 3 2"
example_answer = " 1"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '5', ' 4', ' 3', ' 2']
Tokenized answer: [' 1']


Performance on answer token:
Rank: 0        Logit: 17.47 Prob: 48.35% Token: | 1|

Top 0th token. Logit: 17.47 Prob: 48.35% Token: | 1|
Top 1th token. Logit: 16.10 Prob: 12.34% Token: | 2|
Top 2th token. Logit: 15.90 Prob: 10.12% Token: | 0|
Top 3th token. Logit: 15.43 Prob:  6.31% Token: | 3|
Top 4th token. Logit: 14.73 Prob:  3.14% Token: |
|
Top 5th token. Logit: 14.32 Prob:  2.07% Token: |.|
Top 6th token. Logit: 13.90 Prob:  1.36% Token: | 5|
Top 7th token. Logit: 13.82 Prob:  1.26% Token: | 4|
Top 8th token. Logit: 13.80 Prob:  1.24% Token: | .|
Top 9th token. Logit: 13.48 Prob:  0.90% Token: | -|


Ranks of the answer tokens: [(' 1', 0)]

In [ ]:
example_prompt = "15 14 13 12"
example_answer = " 11"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '15', ' 14', ' 13', ' 12']
Tokenized answer: [' 11']


Performance on answer token:
Rank: 0        Logit: 17.71 Prob: 50.49% Token: | 11|

Top 0th token. Logit: 17.71 Prob: 50.49% Token: | 11|
Top 1th token. Logit: 16.27 Prob: 11.95% Token: | 12|
Top 2th token. Logit: 15.88 Prob:  8.07% Token: | 10|
Top 3th token. Logit: 15.02 Prob:  3.43% Token: | 13|
Top 4th token. Logit: 14.99 Prob:  3.30% Token: | 9|
Top 5th token. Logit: 14.53 Prob:  2.08% Token: |
|
Top 6th token. Logit: 14.16 Prob:  1.44% Token: | 7|
Top 7th token. Logit: 14.10 Prob:  1.35% Token: |-|
Top 8th token. Logit: 13.85 Prob:  1.06% Token: |/|
Top 9th token. Logit: 13.73 Prob:  0.94% Token: | 8|


Ranks of the answer tokens: [(' 11', 0)]

In [ ]:
example_prompt = " 21 20 19 18 17 16 15 14 13 12"
example_answer = " 11"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', ' 21', ' 20', ' 19', ' 18', ' 17', ' 16', ' 15', ' 14', ' 13', ' 12']
Tokenized answer: [' 11']


Performance on answer token:
Rank: 0        Logit: 23.43 Prob: 96.63% Token: | 11|

Top 0th token. Logit: 23.43 Prob: 96.63% Token: | 11|
Top 1th token. Logit: 19.30 Prob:  1.55% Token: | 12|
Top 2th token. Logit: 18.46 Prob:  0.67% Token: | 13|
Top 3th token. Logit: 17.51 Prob:  0.26% Token: | November|
Top 4th token. Logit: 16.87 Prob:  0.14% Token: | 10|
Top 5th token. Logit: 15.89 Prob:  0.05% Token: | Ten|
Top 6th token. Logit: 15.84 Prob:  0.05% Token: |
|
Top 7th token. Logit: 15.73 Prob:  0.04% Token: | 2011|
Top 8th token. Logit: 15.38 Prob:  0.03% Token: | Nov|
Top 9th token. Logit: 15.27 Prob:  0.03% Token: | 9|


Ranks of the answer tokens: [(' 11', 0)]

In [ ]:
example_prompt = " 21 20 19 18 17 16 15 14 13 13"
example_answer = " 11"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', ' 21', ' 20', ' 19', ' 18', ' 17', ' 16', ' 15', ' 14', ' 13', ' 13']
Tokenized answer: [' 11']


Performance on answer token:
Rank: 1        Logit: 17.03 Prob:  1.59% Token: | 11|

Top 0th token. Logit: 21.11 Prob: 94.03% Token: | 12|
Top 1th token. Logit: 17.03 Prob:  1.59% Token: | 11|
Top 2th token. Logit: 16.49 Prob:  0.93% Token: | 13|
Top 3th token. Logit: 15.77 Prob:  0.45% Token: | 2012|
Top 4th token. Logit: 15.65 Prob:  0.40% Token: | 14|
Top 5th token. Logit: 15.23 Prob:  0.26% Token: |
|
Top 6th token. Logit: 14.58 Prob:  0.14% Token: | 10|
Top 7th token. Logit: 14.37 Prob:  0.11% Token: | -|
Top 8th token. Logit: 13.92 Prob:  0.07% Token: | December|
Top 9th token. Logit: 13.65 Prob:  0.05% Token: |

|


Ranks of the answer tokens: [(' 11', 1)]

In [ ]:
example_prompt = " 12 11 10 9 8 7 6 5 4 3"
example_answer = " 2"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', ' 12', ' 11', ' 10', ' 9', ' 8', ' 7', ' 6', ' 5', ' 4', ' 3']
Tokenized answer: [' 2']


Performance on answer token:
Rank: 0        Logit: 19.88 Prob: 94.48% Token: | 2|

Top 0th token. Logit: 19.88 Prob: 94.48% Token: | 2|
Top 1th token. Logit: 15.11 Prob:  0.80% Token: | 3|
Top 2th token. Logit: 15.10 Prob:  0.80% Token: | 1|
Top 3th token. Logit: 14.54 Prob:  0.45% Token: | 4|
Top 4th token. Logit: 13.98 Prob:  0.26% Token: |.|
Top 5th token. Logit: 13.88 Prob:  0.23% Token: |
|
Top 6th token. Logit: 13.87 Prob:  0.23% Token: | 12|
Top 7th token. Logit: 13.61 Prob:  0.18% Token: | 0|
Top 8th token. Logit: 13.12 Prob:  0.11% Token: | -|
Top 9th token. Logit: 13.00 Prob:  0.10% Token: | 02|


Ranks of the answer tokens: [(' 2', 0)]

In [ ]:
example_prompt = " 12 11 10 9 8 7 6 5 4 4"
example_answer = " 2"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', ' 12', ' 11', ' 10', ' 9', ' 8', ' 7', ' 6', ' 5', ' 4', ' 4']
Tokenized answer: [' 2']


Performance on answer token:
Rank: 2        Logit: 15.07 Prob:  2.86% Token: | 2|

Top 0th token. Logit: 18.44 Prob: 82.86% Token: | 3|
Top 1th token. Logit: 15.87 Prob:  6.36% Token: | 4|
Top 2th token. Logit: 15.07 Prob:  2.86% Token: | 2|
Top 3th token. Logit: 13.92 Prob:  0.91% Token: | 5|
Top 4th token. Logit: 13.51 Prob:  0.60% Token: | 1|
Top 5th token. Logit: 13.41 Prob:  0.54% Token: | 0|
Top 6th token. Logit: 13.11 Prob:  0.40% Token: |
|
Top 7th token. Logit: 12.93 Prob:  0.34% Token: |.|
Top 8th token. Logit: 12.51 Prob:  0.22% Token: | 11|
Top 9th token. Logit: 12.25 Prob:  0.17% Token: | -|


Ranks of the answer tokens: [(' 2', 2)]

# Numeric

In [ ]:
example_prompt = "1 2 3 4"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 3', ' 4']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 18.76 Prob: 96.18% Token: | 5|

Top 0th token. Logit: 18.76 Prob: 96.18% Token: | 5|
Top 1th token. Logit: 13.27 Prob:  0.40% Token: | Next|
Top 2th token. Logit: 13.01 Prob:  0.30% Token: |
|
Top 3th token. Logit: 12.87 Prob:  0.27% Token: | >|
Top 4th token. Logit: 12.04 Prob:  0.12% Token: | 4|
Top 5th token. Logit: 11.88 Prob:  0.10% Token: | 50|
Top 6th token. Logit: 11.83 Prob:  0.09% Token: | 6|
Top 7th token. Logit: 11.71 Prob:  0.08% Token: | <|
Top 8th token. Logit: 11.64 Prob:  0.08% Token: | $|
Top 9th token. Logit: 11.63 Prob:  0.08% Token: | 1|


Ranks of the answer tokens: [(' 5', 0)]

In [ ]:
example_prompt = "15 16 17 18"
example_answer = " 19"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '15', ' 16', ' 17', ' 18']
Tokenized answer: [' 19']


Performance on answer token:
Rank: 0        Logit: 18.12 Prob: 91.71% Token: | 19|

Top 0th token. Logit: 18.12 Prob: 91.71% Token: | 19|
Top 1th token. Logit: 14.44 Prob:  2.32% Token: |
|
Top 2th token. Logit: 13.18 Prob:  0.66% Token: | 20|
Top 3th token. Logit: 12.93 Prob:  0.51% Token: | 18|
Top 4th token. Logit: 12.10 Prob:  0.22% Token: | 21|
Top 5th token. Logit: 11.90 Prob:  0.18% Token: |19|
Top 6th token. Logit: 11.38 Prob:  0.11% Token: | 15|
Top 7th token. Logit: 11.37 Prob:  0.11% Token: | +|
Top 8th token. Logit: 11.35 Prob:  0.11% Token: | -|
Top 9th token. Logit: 11.33 Prob:  0.10% Token: | 1|


Ranks of the answer tokens: [(' 19', 0)]

In [ ]:
example_prompt = "1 2 3 4 5 6 7 8 9 10 11 12 13 14"
example_answer = " 15"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9', ' 10', ' 11', ' 12', ' 13', ' 14']
Tokenized answer: [' 15']


Performance on answer token:
Rank: 0        Logit: 20.43 Prob: 98.98% Token: | 15|

Top 0th token. Logit: 20.43 Prob: 98.98% Token: | 15|
Top 1th token. Logit: 13.78 Prob:  0.13% Token: |15|
Top 2th token. Logit: 13.08 Prob:  0.06% Token: |
|
Top 3th token. Logit: 12.67 Prob:  0.04% Token: | 150|
Top 4th token. Logit: 12.61 Prob:  0.04% Token: | 2015|
Top 5th token. Logit: 12.60 Prob:  0.04% Token: | 5|
Top 6th token. Logit: 12.59 Prob:  0.04% Token: | #|
Top 7th token. Logit: 12.38 Prob:  0.03% Token: | 25|
Top 8th token. Logit: 12.35 Prob:  0.03% Token: | 1|
Top 9th token. Logit: 12.33 Prob:  0.03% Token: | 16|


Ranks of the answer tokens: [(' 15', 0)]

## Corruption by repeat or switching

In [ ]:
example_prompt = "1 2 4 3"
example_answer = " 4"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 4', ' 3']
Tokenized answer: [' 4']


Performance on answer token:
Rank: 0        Logit: 16.02 Prob: 52.81% Token: | 4|

Top 0th token. Logit: 16.02 Prob: 52.81% Token: | 4|
Top 1th token. Logit: 15.34 Prob: 26.86% Token: | 5|
Top 2th token. Logit: 12.70 Prob:  1.92% Token: | 6|
Top 3th token. Logit: 12.51 Prob:  1.58% Token: | Next|
Top 4th token. Logit: 12.43 Prob:  1.46% Token: |
|
Top 5th token. Logit: 11.89 Prob:  0.85% Token: | >|
Top 6th token. Logit: 11.87 Prob:  0.84% Token: | 1|
Top 7th token. Logit: 11.60 Prob:  0.64% Token: | -|
Top 8th token. Logit: 11.27 Prob:  0.46% Token: | ||
Top 9th token. Logit: 10.94 Prob:  0.33% Token: | 8|


Ranks of the answer tokens: [(' 4', 0)]

In [ ]:
example_prompt = "15 16 18 17"
example_answer = " 19"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '15', ' 16', ' 18', ' 17']
Tokenized answer: [' 19']


Performance on answer token:
Rank: 6        Logit: 15.96 Prob:  2.40% Token: | 19|

Top 0th token. Logit: 18.15 Prob: 21.47% Token: | 16|
Top 1th token. Logit: 17.90 Prob: 16.74% Token: | 18|
Top 2th token. Logit: 17.41 Prob: 10.24% Token: | 17|
Top 3th token. Logit: 17.14 Prob:  7.80% Token: | 14|
Top 4th token. Logit: 16.95 Prob:  6.46% Token: | 15|
Top 5th token. Logit: 16.63 Prob:  4.68% Token: | 12|
Top 6th token. Logit: 15.96 Prob:  2.40% Token: | 19|
Top 7th token. Logit: 15.90 Prob:  2.25% Token: | 13|
Top 8th token. Logit: 15.83 Prob:  2.11% Token: | 8|
Top 9th token. Logit: 15.57 Prob:  1.64% Token: | 11|


Ranks of the answer tokens: [(' 19', 6)]

In [ ]:
example_prompt = "15 16 17 17"
example_answer = " 19"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '15', ' 16', ' 17', ' 17']
Tokenized answer: [' 19']


Performance on answer token:
Rank: 4        Logit: 13.45 Prob:  3.00% Token: | 19|

Top 0th token. Logit: 16.31 Prob: 52.38% Token: | 18|
Top 1th token. Logit: 15.25 Prob: 18.14% Token: | 17|
Top 2th token. Logit: 13.83 Prob:  4.37% Token: |
|
Top 3th token. Logit: 13.69 Prob:  3.81% Token: | 16|
Top 4th token. Logit: 13.45 Prob:  3.00% Token: | 19|
Top 5th token. Logit: 12.55 Prob:  1.21% Token: | 14|
Top 6th token. Logit: 11.77 Prob:  0.56% Token: | 0|
Top 7th token. Logit: 11.76 Prob:  0.55% Token: | 20|
Top 8th token. Logit: 11.59 Prob:  0.47% Token: | 15|
Top 9th token. Logit: 11.39 Prob:  0.38% Token: | 21|


Ranks of the answer tokens: [(' 19', 4)]

## Generate overlapping

In [ ]:
prompts = [" ".join(map(str, range(i, i+4))) for i in range(1, 21)]
answers = [(str(i+4), str(i+3)) for i in range(1, 21)]
corrupted_prompts = [" ".join(map(str, list(range(i, i+3)) + [i+2])) for i in range(1, 21)]

In [ ]:
for i, prompt in enumerate(prompts):
    utils.test_prompt(prompt, answers[i][0], model, prepend_bos=True, top_k=3)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 3', ' 4']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 18.76 Prob: 96.18% Token: | 5|

Top 0th token. Logit: 18.76 Prob: 96.18% Token: | 5|
Top 1th token. Logit: 13.27 Prob:  0.40% Token: | Next|
Top 2th token. Logit: 13.01 Prob:  0.30% Token: |
|


Ranks of the answer tokens: [(' 5', 0)]

Tokenized prompt: ['<|endoftext|>', '2', ' 3', ' 4', ' 5']
Tokenized answer: [' 6']


...

Ranks of the answer tokens: [(' 23', 0)]

Tokenized prompt: ['<|endoftext|>', '20', ' 21', ' 22', ' 23']
Tokenized answer: [' 24']


Performance on answer token:
Rank: 0        Logit: 17.52 Prob: 81.52% Token: | 24|

Top 0th token. Logit: 17.52 Prob: 81.52% Token: | 24|
Top 1th token. Logit: 14.48 Prob:  3.89% Token: |
|
Top 2th token. Logit: 13.54 Prob:  1.53% Token: | 25|


Ranks of the answer tokens: [(' 24', 0)]

In [ ]:
for i, prompt in enumerate(corrupted_prompts):
    utils.test_prompt(prompt, answers[i][1], model, prepend_bos=True, top_k=3)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 3', ' 3']
Tokenized answer: [' 4']


Performance on answer token:
Rank: 0        Logit: 16.84 Prob: 87.91% Token: | 4|

Top 0th token. Logit: 16.84 Prob: 87.91% Token: | 4|
Top 1th token. Logit: 13.00 Prob:  1.89% Token: |
|
Top 2th token. Logit: 11.96 Prob:  0.67% Token: | 5|


Ranks of the answer tokens: [(' 4', 0)]

Tokenized prompt: ['<|endoftext|>', '2', ' 3', ' 4', ' 4']
Tokenized answer: [' 5']


...

Ranks of the answer tokens: [(' 22', 0)]

Tokenized prompt: ['<|endoftext|>', '20', ' 21', ' 22', ' 22']
Tokenized answer: [' 23']


Performance on answer token:
Rank: 0        Logit: 17.12 Prob: 72.28% Token: | 23|

Top 0th token. Logit: 17.12 Prob: 72.28% Token: | 23|
Top 1th token. Logit: 14.73 Prob:  6.66% Token: | 22|
Top 2th token. Logit: 14.01 Prob:  3.23% Token: |
|


Ranks of the answer tokens: [(' 23', 0)]

## Distinguish Additive vs Multiplicative

In [ ]:
example_prompt = "2 4 6 8"
example_answer = " 10"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 6', ' 8']
Tokenized answer: [' 10']


Performance on answer token:
Rank: 1        Logit: 13.86 Prob:  5.65% Token: | 10|

Top 0th token. Logit: 16.10 Prob: 53.36% Token: | 9|
Top 1th token. Logit: 13.86 Prob:  5.65% Token: | 10|
Top 2th token. Logit: 13.61 Prob:  4.42% Token: |
|
Top 3th token. Logit: 13.58 Prob:  4.30% Token: | 8|
Top 4th token. Logit: 13.23 Prob:  3.01% Token: | 11|
Top 5th token. Logit: 13.05 Prob:  2.52% Token: | 5|
Top 6th token. Logit: 13.02 Prob:  2.44% Token: | 7|
Top 7th token. Logit: 12.72 Prob:  1.81% Token: | 1|
Top 8th token. Logit: 12.72 Prob:  1.81% Token: | 13|
Top 9th token. Logit: 12.51 Prob:  1.46% Token: | 3|


Ranks of the answer tokens: [(' 10', 1)]

In [ ]:
example_prompt = "0 2 4 6 8 10"
example_answer = " 12"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ' 2', ' 4', ' 6', ' 8', ' 10']
Tokenized answer: [' 12']


Performance on answer token:
Rank: 1        Logit: 16.25 Prob: 17.95% Token: | 12|

Top 0th token. Logit: 16.70 Prob: 28.31% Token: |
|
Top 1th token. Logit: 16.25 Prob: 17.95% Token: | 12|
Top 2th token. Logit: 15.79 Prob: 11.38% Token: | 11|
Top 3th token. Logit: 15.14 Prob:  5.95% Token: | 13|
Top 4th token. Logit: 15.02 Prob:  5.25% Token: | 15|
Top 5th token. Logit: 14.30 Prob:  2.56% Token: | 14|
Top 6th token. Logit: 13.80 Prob:  1.54% Token: | 20|
Top 7th token. Logit: 13.67 Prob:  1.35% Token: | 9|
Top 8th token. Logit: 13.61 Prob:  1.28% Token: | 0|
Top 9th token. Logit: 13.47 Prob:  1.11% Token: | 10|


Ranks of the answer tokens: [(' 12', 1)]

In [ ]:
example_prompt = "2 4 6 8 10 12"
example_answer = " 14"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 6', ' 8', ' 10', ' 12']
Tokenized answer: [' 14']


Performance on answer token:
Rank: 0        Logit: 18.26 Prob: 35.59% Token: | 14|

Top 0th token. Logit: 18.26 Prob: 35.59% Token: | 14|
Top 1th token. Logit: 17.97 Prob: 26.59% Token: | 13|
Top 2th token. Logit: 17.67 Prob: 19.57% Token: | 15|
Top 3th token. Logit: 15.95 Prob:  3.53% Token: | 16|
Top 4th token. Logit: 15.35 Prob:  1.92% Token: | 17|
Top 5th token. Logit: 15.29 Prob:  1.82% Token: | 18|
Top 6th token. Logit: 15.25 Prob:  1.75% Token: | 12|
Top 7th token. Logit: 15.21 Prob:  1.68% Token: |
|
Top 8th token. Logit: 14.82 Prob:  1.14% Token: | 19|
Top 9th token. Logit: 13.83 Prob:  0.42% Token: | 20|


Ranks of the answer tokens: [(' 14', 0)]

Multiplicative

In [ ]:
example_prompt = "2 4 8 16"
example_answer = " 32"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 8', ' 16']
Tokenized answer: [' 32']


Performance on answer token:
Rank: 26       Logit: 11.27 Prob:  0.77% Token: | 32|

Top 0th token. Logit: 13.81 Prob:  9.74% Token: |
|
Top 1th token. Logit: 13.36 Prob:  6.22% Token: | 16|
Top 2th token. Logit: 12.97 Prob:  4.20% Token: | 17|
Top 3th token. Logit: 12.93 Prob:  4.04% Token: | 8|
Top 4th token. Logit: 12.72 Prob:  3.29% Token: | 0|
Top 5th token. Logit: 12.62 Prob:  2.96% Token: | 18|
Top 6th token. Logit: 12.55 Prob:  2.77% Token: | 9|
Top 7th token. Logit: 12.51 Prob:  2.66% Token: | 1|
Top 8th token. Logit: 12.45 Prob:  2.49% Token: | 4|
Top 9th token. Logit: 12.40 Prob:  2.38% Token: | 15|


Ranks of the answer tokens: [(' 32', 26)]

In [ ]:
example_prompt = "1 2 4 8 16 32"
example_answer = " 64"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 4', ' 8', ' 16', ' 32']
Tokenized answer: [' 64']


Performance on answer token:
Rank: 2        Logit: 15.96 Prob:  7.68% Token: | 64|

Top 0th token. Logit: 16.42 Prob: 12.21% Token: | 33|
Top 1th token. Logit: 16.18 Prob:  9.64% Token: | 34|
Top 2th token. Logit: 15.96 Prob:  7.68% Token: | 64|
Top 3th token. Logit: 15.83 Prob:  6.76% Token: | 32|
Top 4th token. Logit: 15.53 Prob:  5.02% Token: |
|
Top 5th token. Logit: 15.41 Prob:  4.46% Token: | 36|
Top 6th token. Logit: 15.40 Prob:  4.42% Token: | 48|
Top 7th token. Logit: 15.36 Prob:  4.21% Token: | 49|
Top 8th token. Logit: 14.77 Prob:  2.35% Token: | 43|
Top 9th token. Logit: 14.63 Prob:  2.03% Token: | 35|


Ranks of the answer tokens: [(' 64', 2)]

In [ ]:
example_prompt = "2 4 8 16 32"
example_answer = " 64"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 8', ' 16', ' 32']
Tokenized answer: [' 64']


Performance on answer token:
Rank: 3        Logit: 13.63 Prob:  2.83% Token: | 64|

Top 0th token. Logit: 14.60 Prob:  7.42% Token: | 32|
Top 1th token. Logit: 14.57 Prob:  7.23% Token: |
|
Top 2th token. Logit: 13.68 Prob:  2.97% Token: | 36|
Top 3th token. Logit: 13.63 Prob:  2.83% Token: | 64|
Top 4th token. Logit: 13.60 Prob:  2.76% Token: | 16|
Top 5th token. Logit: 13.56 Prob:  2.62% Token: | 33|
Top 6th token. Logit: 13.38 Prob:  2.20% Token: | 34|
Top 7th token. Logit: 13.37 Prob:  2.18% Token: | 48|
Top 8th token. Logit: 13.20 Prob:  1.83% Token: | 0|
Top 9th token. Logit: 13.15 Prob:  1.75% Token: | 40|


Ranks of the answer tokens: [(' 64', 3)]

In [ ]:
example_prompt = "2 4 8 16 32 64"
example_answer = " 128"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 8', ' 16', ' 32', ' 64']
Tokenized answer: [' 128']


Performance on answer token:
Rank: 2        Logit: 14.17 Prob:  2.42% Token: | 128|

Top 0th token. Logit: 15.68 Prob: 10.99% Token: | 64|
Top 1th token. Logit: 15.16 Prob:  6.53% Token: |
|
Top 2th token. Logit: 14.17 Prob:  2.42% Token: | 128|
Top 3th token. Logit: 14.14 Prob:  2.36% Token: | 72|
Top 4th token. Logit: 14.07 Prob:  2.20% Token: | 32|
Top 5th token. Logit: 14.04 Prob:  2.13% Token: | 48|
Top 6th token. Logit: 14.02 Prob:  2.09% Token: | 96|
Top 7th token. Logit: 13.99 Prob:  2.02% Token: | 68|
Top 8th token. Logit: 13.76 Prob:  1.61% Token: | 4|
Top 9th token. Logit: 13.74 Prob:  1.58% Token: | 100|


Ranks of the answer tokens: [(' 128', 2)]

# Number words

In [ ]:
example_prompt = "one two"
example_answer = " three"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'one', ' two']
Tokenized answer: [' three']


Performance on answer token:
Rank: 0        Logit: 13.85 Prob: 31.08% Token: | three|

Top 0th token. Logit: 13.85 Prob: 31.08% Token: | three|
Top 1th token. Logit: 12.35 Prob:  6.96% Token: | two|
Top 2th token. Logit: 12.05 Prob:  5.13% Token: | four|
Top 3th token. Logit: 11.98 Prob:  4.80% Token: |,|
Top 4th token. Logit: 11.57 Prob:  3.18% Token: |-|
Top 5th token. Logit: 11.35 Prob:  2.56% Token: |
|
Top 6th token. Logit: 11.12 Prob:  2.03% Token: | five|
Top 7th token. Logit: 11.02 Prob:  1.83% Token: | one|
Top 8th token. Logit: 10.74 Prob:  1.39% Token: |.|
Top 9th token. Logit: 10.62 Prob:  1.23% Token: | or|


Ranks of the answer tokens: [(' three', 0)]

In [ ]:
example_prompt = "two three four five"
example_answer = " six"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'two', ' three', ' four', ' five']
Tokenized answer: [' six']


Performance on answer token:
Rank: 0        Logit: 15.49 Prob: 59.69% Token: | six|

Top 0th token. Logit: 15.49 Prob: 59.69% Token: | six|
Top 1th token. Logit: 12.70 Prob:  3.68% Token: |
|
Top 2th token. Logit: 12.51 Prob:  3.03% Token: | seven|
Top 3th token. Logit: 12.14 Prob:  2.10% Token: | five|
Top 4th token. Logit: 12.10 Prob:  2.01% Token: | one|
Top 5th token. Logit: 11.61 Prob:  1.24% Token: | 6|
Top 6th token. Logit: 11.42 Prob:  1.02% Token: | four|
Top 7th token. Logit: 11.21 Prob:  0.83% Token: | eight|
Top 8th token. Logit: 11.02 Prob:  0.69% Token: | ...|
Top 9th token. Logit: 11.01 Prob:  0.68% Token: | ten|


Ranks of the answer tokens: [(' six', 0)]

In [ ]:
example_prompt = "two four six eight ten twelve"
example_answer = " fourteen"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'two', ' four', ' six', ' eight', ' ten', ' twelve']
Tokenized answer: [' fourteen']


Performance on answer token:
Rank: 1        Logit: 15.32 Prob: 12.08% Token: | fourteen|

Top 0th token. Logit: 16.36 Prob: 34.27% Token: | thirteen|
Top 1th token. Logit: 15.32 Prob: 12.08% Token: | fourteen|
Top 2th token. Logit: 15.28 Prob: 11.63% Token: | twelve|
Top 3th token. Logit: 14.67 Prob:  6.34% Token: | fifteen|
Top 4th token. Logit: 14.67 Prob:  6.30% Token: | sixteen|
Top 5th token. Logit: 13.84 Prob:  2.76% Token: | eighteen|
Top 6th token. Logit: 13.61 Prob:  2.19% Token: | seventeen|
Top 7th token. Logit: 13.54 Prob:  2.04% Token: | twenty|
Top 8th token. Logit: 13.45 Prob:  1.86% Token: | eleven|
Top 9th token. Logit: 13.41 Prob:  1.79% Token: | one|


Ranks of the answer tokens: [(' fourteen', 1)]

In [ ]:
example_prompt = "two four six eight ten"
example_answer = " twelve"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'two', ' four', ' six', ' eight', ' ten']
Tokenized answer: [' twelve']


Performance on answer token:
Rank: 1        Logit: 14.03 Prob: 12.50% Token: | twelve|

Top 0th token. Logit: 14.30 Prob: 16.38% Token: | eleven|
Top 1th token. Logit: 14.03 Prob: 12.50% Token: | twelve|
Top 2th token. Logit: 13.33 Prob:  6.21% Token: | nine|
Top 3th token. Logit: 12.88 Prob:  3.98% Token: | seven|
Top 4th token. Logit: 12.85 Prob:  3.86% Token: | ten|
Top 5th token. Logit: 12.74 Prob:  3.47% Token: | thirteen|
Top 6th token. Logit: 12.68 Prob:  3.25% Token: |
|
Top 7th token. Logit: 12.54 Prob:  2.83% Token: | fifteen|
Top 8th token. Logit: 12.48 Prob:  2.67% Token: | twenty|
Top 9th token. Logit: 12.48 Prob:  2.66% Token: | sixteen|


Ranks of the answer tokens: [(' twelve', 1)]

In [ ]:
example_prompt = "two four six eight"
example_answer = " ten"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'two', ' four', ' six', ' eight']
Tokenized answer: [' ten']


Performance on answer token:
Rank: 3        Logit: 12.06 Prob:  3.62% Token: | ten|

Top 0th token. Logit: 13.75 Prob: 19.55% Token: | nine|
Top 1th token. Logit: 13.09 Prob: 10.10% Token: | seven|
Top 2th token. Logit: 13.03 Prob:  9.53% Token: | eight|
Top 3th token. Logit: 12.06 Prob:  3.62% Token: | ten|
Top 4th token. Logit: 11.81 Prob:  2.81% Token: | six|
Top 5th token. Logit: 11.79 Prob:  2.75% Token: | five|
Top 6th token. Logit: 11.65 Prob:  2.39% Token: | eleven|
Top 7th token. Logit: 11.61 Prob:  2.29% Token: |
|
Top 8th token. Logit: 11.59 Prob:  2.26% Token: | twelve|
Top 9th token. Logit: 11.48 Prob:  2.02% Token: | three|


Ranks of the answer tokens: [(' ten', 3)]

In [ ]:
example_prompt = "fourteen fifteen sixteen seventeen"
example_answer = " eighteen"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'four', 'teen', ' fifteen', ' sixteen', ' seventeen']
Tokenized answer: [' eighteen']


Performance on answer token:
Rank: 0        Logit: 16.03 Prob: 33.27% Token: | eighteen|

Top 0th token. Logit: 16.03 Prob: 33.27% Token: | eighteen|
Top 1th token. Logit: 15.15 Prob: 13.78% Token: | twenty|
Top 2th token. Logit: 15.04 Prob: 12.45% Token: | sixteen|
Top 3th token. Logit: 14.95 Prob: 11.37% Token: | seventeen|
Top 4th token. Logit: 13.93 Prob:  4.09% Token: | nineteen|
Top 5th token. Logit: 13.06 Prob:  1.72% Token: | 18|
Top 6th token. Logit: 13.01 Prob:  1.63% Token: | fourteen|
Top 7th token. Logit: 12.64 Prob:  1.13% Token: | thirteen|
Top 8th token. Logit: 12.62 Prob:  1.10% Token: | thirty|
Top 9th token. Logit: 12.45 Prob:  0.93% Token: | fifteen|


Ranks of the answer tokens: [(' eighteen', 0)]

Once it gets to numbers with more than 1 token, hard to assess, so try that in future work. Whether split by space, no space, or dash- no result. Future work can try scaling it up to more elems in seq.

In [ ]:
example_prompt = "thirtyeight thirtynine fourty fourtyone"
example_answer = " fourtytwo"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'th', 'irty', 'eight', ' thirty', 'nine', ' four', 'ty', ' four', 'ty', 'one']
Tokenized answer: [' four', 'ty', 'two']


Performance on answer token:
Rank: 3        Logit:  9.09 Prob:  1.24% Token: | four|

Top 0th token. Logit: 10.60 Prob:  5.63% Token: |
|
Top 1th token. Logit:  9.85 Prob:  2.66% Token: |.|
Top 2th token. Logit:  9.25 Prob:  1.46% Token: |,|
Top 3th token. Logit:  9.09 Prob:  1.24% Token: | four|
Top 4th token. Logit:  8.96 Prob:  1.09% Token: | twenty|
Top 5th token. Logit:  8.87 Prob:  1.00% Token: | forty|
Top 6th token. Logit:  8.85 Prob:  0.98% Token: | one|
Top 7th token. Logit:  8.73 Prob:  0.86% Token: | five|
Top 8th token. Logit:  8.68 Prob:  0.83% Token: | (|
Top 9th token. Logit:  8.63 Prob:  0.79% Token: | three|


Performance on answer token:
Rank: 0        Logit: 14.70 Prob: 61.60% Token: |ty|

Top 0th token. Logit: 14.70 Prob: 61.60% Token: |ty|
Top 1th token. Logit: 11.34 Prob:  2.13% Token: |teen|
Top 2th token. Logit: 11.28 Prob:  2.01% Token: |th|
Top 3th token. Logit: 11.04 Prob:  1.58% Token: |four|
Top 4th token. Logit: 10.90 Prob:  1.37% Token: |-|
Top 5th token. Logit: 10.79 Prob:  1.23% Token: |t|
Top 6th token. Logit: 10.24 Prob:  0.71% Token: |.|
Top 7th token. Logit: 10.05 Prob:  0.59% Token: |y|
Top 8th token. Logit:  9.93 Prob:  0.52% Token: |nine|
Top 9th token. Logit:  9.74 Prob:  0.43% Token: |,|


Performance on answer token:
Rank: 1        Logit: 13.89 Prob:  8.53% Token: |two|

Top 0th token. Logit: 15.71 Prob: 53.03% Token: |one|
Top 1th token. Logit: 13.89 Prob:  8.53% Token: |two|
Top 2th token. Logit: 13.65 Prob:  6.76% Token: |four|
Top 3th token. Logit: 13.63 Prob:  6.60% Token: |five|
Top 4th token. Logit: 12.87 Prob:  3.09% Token: | one|
Top 5th token. Logit: 12.85 Prob:  3.02% Token: |three|
Top 6th token. Logit: 12.68 Prob:  2.55% Token: |seven|
Top 7th token. Logit: 12.55 Prob:  2.23% Token: |eight|
Top 8th token. Logit: 11.98 Prob:  1.27% Token: |six|
Top 9th token. Logit: 11.83 Prob:  1.10% Token: |nine|


Ranks of the answer tokens: [(' four', 3), ('ty', 0), ('two', 1)]

## Genereate overlapping

In [ ]:
!pip install inflect

In [ ]:
import inflect

p = inflect.engine()

prompts = [" ".join(map(lambda x: p.number_to_words(x), range(i, i+4))) for i in range(1, 21)]
answers = [(p.number_to_words(i+4), p.number_to_words(i+3)) for i in range(1, 21)]
corrupted_prompts = [" ".join(map(lambda x: p.number_to_words(x), list(range(i, i+3)) + [i+2])) for i in range(1, 21)]

In [ ]:
for i, prompt in enumerate(prompts):
    utils.test_prompt(prompt, answers[i][0], model, prepend_bos=True, top_k=3)

Tokenized prompt: ['<|endoftext|>', 'one', ' two', ' three', ' four']
Tokenized answer: [' five']


Performance on answer token:
Rank: 0        Logit: 15.29 Prob: 48.87% Token: | five|

Top 0th token. Logit: 15.29 Prob: 48.87% Token: | five|
Top 1th token. Logit: 13.46 Prob:  7.81% Token: | four|
Top 2th token. Logit: 12.81 Prob:  4.07% Token: |
|


Ranks of the answer tokens: [(' five', 0)]

Tokenized prompt: ['<|endoftext|>', 'two', ' three', ' four', ' five']
Tokenized answer: [' six']


...

Performance on answer token:
Rank: 0        Logit: 17.94 Prob: 95.85% Token: |-|

Top 0th token. Logit: 17.94 Prob: 95.85% Token: |-|
Top 1th token. Logit: 12.56 Prob:  0.44% Token: | one|
Top 2th token. Logit: 11.90 Prob:  0.23% Token: | four|


Performance on answer token:
Rank: 0        Logit: 19.16 Prob: 39.82% Token: |four|

Top 0th token. Logit: 19.16 Prob: 39.82% Token: |four|
Top 1th token. Logit: 18.05 Prob: 13.21% Token: |one|
Top 2th token. Logit: 17.90 Prob: 11.34% Token: |five|


Ranks of the answer tokens: [(' twenty', 0), ('-', 0), ('four', 0)]

In [ ]:
for i, prompt in enumerate(corrupted_prompts):
    utils.test_prompt(prompt, answers[i][1], model, prepend_bos=True, top_k=3)

Tokenized prompt: ['<|endoftext|>', 'one', ' two', ' three', ' three']
Tokenized answer: [' four']


Performance on answer token:
Rank: 0        Logit: 16.17 Prob: 69.59% Token: | four|

Top 0th token. Logit: 16.17 Prob: 69.59% Token: | four|
Top 1th token. Logit: 13.01 Prob:  2.94% Token: |
|
Top 2th token. Logit: 12.60 Prob:  1.95% Token: | one|


Ranks of the answer tokens: [(' four', 0)]

Tokenized prompt: ['<|endoftext|>', 'two', ' three', ' four', ' four']
Tokenized answer: [' five']


...

Performance on answer token:
Rank: 0        Logit: 17.82 Prob: 96.88% Token: |-|

Top 0th token. Logit: 17.82 Prob: 96.88% Token: |-|
Top 1th token. Logit: 11.79 Prob:  0.23% Token: | one|
Top 2th token. Logit: 11.41 Prob:  0.16% Token: | two|


Performance on answer token:
Rank: 2        Logit: 17.04 Prob:  8.49% Token: |three|

Top 0th token. Logit: 18.81 Prob: 49.51% Token: |two|
Top 1th token. Logit: 18.36 Prob: 31.79% Token: |one|
Top 2th token. Logit: 17.04 Prob:  8.49% Token: |three|


Ranks of the answer tokens: [(' twenty', 0), ('-', 0), ('three', 2)]

# Months

In [ ]:
example_prompt = "January February March April"
example_answer = " May"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'January', ' February', ' March', ' April']
Tokenized answer: [' May']


Performance on answer token:
Rank: 0        Logit: 30.05 Prob: 99.87% Token: | May|

Top 0th token. Logit: 30.05 Prob: 99.87% Token: | May|
Top 1th token. Logit: 22.47 Prob:  0.05% Token: | 5|
Top 2th token. Logit: 21.69 Prob:  0.02% Token: | July|
Top 3th token. Logit: 21.12 Prob:  0.01% Token: |
|
Top 4th token. Logit: 21.04 Prob:  0.01% Token: | June|
Top 5th token. Logit: 19.32 Prob:  0.00% Token: | Wh|
Top 6th token. Logit: 19.18 Prob:  0.00% Token: | If|
Top 7th token. Logit: 18.90 Prob:  0.00% Token: | March|
Top 8th token. Logit: 18.79 Prob:  0.00% Token: | April|
Top 9th token. Logit: 18.69 Prob:  0.00% Token: | Please|


Ranks of the answer tokens: [(' May', 0)]

# Month to Digit, in-context

In [ ]:
example_prompt = "Month 1 is January. Month 3 is"
example_answer = " March"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Month', ' 1', ' is', ' January', '.', ' Month', ' 3', ' is']
Tokenized answer: [' March']


Performance on answer token:
Rank: 1        Logit: 18.36 Prob: 13.00% Token: | March|

Top 0th token. Logit: 19.81 Prob: 55.14% Token: | February|
Top 1th token. Logit: 18.36 Prob: 13.00% Token: | March|
Top 2th token. Logit: 17.58 Prob:  5.96% Token: | April|
Top 3th token. Logit: 17.48 Prob:  5.39% Token: | December|
Top 4th token. Logit: 17.42 Prob:  5.07% Token: | January|
Top 5th token. Logit: 16.88 Prob:  2.96% Token: | June|
Top 6th token. Logit: 16.84 Prob:  2.85% Token: | August|
Top 7th token. Logit: 16.37 Prob:  1.77% Token: | May|
Top 8th token. Logit: 16.33 Prob:  1.71% Token: | October|
Top 9th token. Logit: 16.12 Prob:  1.39% Token: | November|


Ranks of the answer tokens: [(' March', 1)]

In [ ]:
example_prompt = "Month 1 is January. Month 4 is April. Month 3 is"
example_answer = " March"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Month', ' 1', ' is', ' January', '.', ' Month', ' 4', ' is', ' April', '.', ' Month', ' 3', ' is']
Tokenized answer: [' March']


Performance on answer token:
Rank: 9        Logit: 15.86 Prob:  1.96% Token: | March|

Top 0th token. Logit: 18.57 Prob: 29.44% Token: | May|
Top 1th token. Logit: 18.02 Prob: 17.07% Token: | July|
Top 2th token. Logit: 18.00 Prob: 16.62% Token: | June|
Top 3th token. Logit: 17.08 Prob:  6.68% Token: | October|
Top 4th token. Logit: 17.07 Prob:  6.57% Token: | November|
Top 5th token. Logit: 17.04 Prob:  6.42% Token: | September|
Top 6th token. Logit: 16.55 Prob:  3.90% Token: | August|
Top 7th token. Logit: 16.37 Prob:  3.26% Token: | December|
Top 8th token. Logit: 16.03 Prob:  2.32% Token: | April|
Top 9th token. Logit: 15.86 Prob:  1.96% Token: | March|


Ranks of the answer tokens: [(' March', 9)]

In [ ]:
example_prompt = "Month 3 is"
example_answer = " March"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Month', ' 3', ' is']
Tokenized answer: [' March']


Performance on answer token:
Rank: 228      Logit:  8.28 Prob:  0.03% Token: | March|

Top 0th token. Logit: 14.03 Prob: 10.38% Token: | here|
Top 1th token. Logit: 13.68 Prob:  7.32% Token: | a|
Top 2th token. Logit: 13.65 Prob:  7.09% Token: | the|
Top 3th token. Logit: 13.32 Prob:  5.10% Token: | now|
Top 4th token. Logit: 12.95 Prob:  3.52% Token: | over|
Top 5th token. Logit: 12.85 Prob:  3.20% Token: | coming|
Top 6th token. Logit: 12.50 Prob:  2.25% Token: | out|
Top 7th token. Logit: 12.39 Prob:  2.02% Token: | upon|
Top 8th token. Logit: 12.20 Prob:  1.67% Token: | finally|
Top 9th token. Logit: 12.19 Prob:  1.64% Token: | going|


Ranks of the answer tokens: [(' March', 228)]

# [ name ] is [ digit ]

In [ ]:
example_prompt = "Adam is 1. Bob is 2. Claire is 3. Don is 4. Eve is"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' 1', '.', ' Bob', ' is', ' 2', '.', ' Claire', ' is', ' 3', '.', ' Don', ' is', ' 4', '.', ' Eve', ' is']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 17.48 Prob: 95.09% Token: | 5|

Top 0th token. Logit: 17.48 Prob: 95.09% Token: | 5|
Top 1th token. Logit: 12.20 Prob:  0.49% Token: | 6|
Top 2th token. Logit: 11.90 Prob:  0.36% Token: | 4|
Top 3th token. Logit: 11.82 Prob:  0.33% Token: | 1|
Top 4th token. Logit: 11.63 Prob:  0.27% Token: | a|
Top 5th token. Logit: 10.86 Prob:  0.13% Token: |
|
Top 6th token. Logit: 10.82 Prob:  0.12% Token: | 7|
Top 7th token. Logit: 10.71 Prob:  0.11% Token: | 10|
Top 8th token. Logit: 10.44 Prob:  0.08% Token: | the|
Top 9th token. Logit: 10.35 Prob:  0.08% Token: | 50|


Ranks of the answer tokens: [(' 5', 0)]

## Genrate overlap

In [ ]:
names = ["Adam", "Bob", "Claire", "Don", "Eve"]

prompts = ["{} is {}. {} is {}. {} is {}. {} is {}. {} is".format(names[0], i, names[1], i+1, names[2], i+2, names[3], i+3, names[4]) for i in range(1, 18)]
answers = [(str(i+4), str(i+3)) for i in range(1, 21)]
corrupted_prompts = ["{} is {}. {} is {}. {} is {}. {} is {}. {} is".format(names[0], i, names[1], i+1, names[2], i+2, names[3], i+2, names[4]) for i in range(1, 18)]

In [ ]:
for i, prompt in enumerate(prompts):
    utils.test_prompt(prompt, answers[i][0], model, prepend_bos=True, top_k=3)

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' 1', '.', ' Bob', ' is', ' 2', '.', ' Claire', ' is', ' 3', '.', ' Don', ' is', ' 4', '.', ' Eve', ' is']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 17.48 Prob: 95.09% Token: | 5|

Top 0th token. Logit: 17.48 Prob: 95.09% Token: | 5|
Top 1th token. Logit: 12.20 Prob:  0.49% Token: | 6|
Top 2th token. Logit: 11.90 Prob:  0.36% Token: | 4|


Ranks of the answer tokens: [(' 5', 0)]

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' 2', '.', ' Bob', ' is', ' 3', '.', ' Claire', ' is', ' 4', '.', ' Don', ' is', ' 5', '.', ' Eve', ' is']
Tokenized answer: [' 6']


...

Ranks of the answer tokens: [(' 20', 0)]

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' 17', '.', ' Bob', ' is', ' 18', '.', ' Claire', ' is', ' 19', '.', ' Don', ' is', ' 20', '.', ' Eve', ' is']
Tokenized answer: [' 21']


Performance on answer token:
Rank: 0        Logit: 18.46 Prob: 91.62% Token: | 21|

Top 0th token. Logit: 18.46 Prob: 91.62% Token: | 21|
Top 1th token. Logit: 14.64 Prob:  2.01% Token: | 22|
Top 2th token. Logit: 14.07 Prob:  1.13% Token: | 20|


Ranks of the answer tokens: [(' 21', 0)]

Have it be similar enough, but don't want correct answer in rank 0:

In [ ]:
for i, prompt in enumerate(corrupted_prompts):
    utils.test_prompt(prompt, answers[i][0], model, prepend_bos=True, top_k=3)

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' 1', '.', ' Bob', ' is', ' 2', '.', ' Claire', ' is', ' 3', '.', ' Don', ' is', ' 3', '.', ' Eve', ' is']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 2        Logit: 14.16 Prob:  5.88% Token: | 5|

Top 0th token. Logit: 16.59 Prob: 66.91% Token: | 4|
Top 1th token. Logit: 14.57 Prob:  8.90% Token: | 3|
Top 2th token. Logit: 14.16 Prob:  5.88% Token: | 5|


Ranks of the answer tokens: [(' 5', 2)]

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' 2', '.', ' Bob', ' is', ' 3', '.', ' Claire', ' is', ' 4', '.', ' Don', ' is', ' 4', '.', ' Eve', ' is']
Tokenized answer: [' 6']


...

Ranks of the answer tokens: [(' 20', 2)]

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' 17', '.', ' Bob', ' is', ' 18', '.', ' Claire', ' is', ' 19', '.', ' Don', ' is', ' 19', '.', ' Eve', ' is']
Tokenized answer: [' 21']


Performance on answer token:
Rank: 2        Logit: 14.34 Prob:  3.37% Token: | 21|

Top 0th token. Logit: 17.22 Prob: 60.39% Token: | 20|
Top 1th token. Logit: 16.17 Prob: 21.14% Token: | 19|
Top 2th token. Logit: 14.34 Prob:  3.37% Token: | 21|


Ranks of the answer tokens: [(' 21', 2)]

Actually, unlike the pure case, the -1 wrong answer is always rank 0. Perhaps giving it more structure gave it better predictability (more patterns indicating what is what, instead of "fenceless" digits between them):

In [ ]:
for i, prompt in enumerate(corrupted_prompts):
    utils.test_prompt(prompt, answers[i][1], model, prepend_bos=True, top_k=3)

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' 1', '.', ' Bob', ' is', ' 2', '.', ' Claire', ' is', ' 3', '.', ' Don', ' is', ' 3', '.', ' Eve', ' is']
Tokenized answer: [' 4']


Performance on answer token:
Rank: 0        Logit: 16.59 Prob: 66.91% Token: | 4|

Top 0th token. Logit: 16.59 Prob: 66.91% Token: | 4|
Top 1th token. Logit: 14.57 Prob:  8.90% Token: | 3|
Top 2th token. Logit: 14.16 Prob:  5.88% Token: | 5|


Ranks of the answer tokens: [(' 4', 0)]

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' 2', '.', ' Bob', ' is', ' 3', '.', ' Claire', ' is', ' 4', '.', ' Don', ' is', ' 4', '.', ' Eve', ' is']
Tokenized answer: [' 5']


...

Ranks of the answer tokens: [(' 19', 0)]

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' 17', '.', ' Bob', ' is', ' 18', '.', ' Claire', ' is', ' 19', '.', ' Don', ' is', ' 19', '.', ' Eve', ' is']
Tokenized answer: [' 20']


Performance on answer token:
Rank: 0        Logit: 17.22 Prob: 60.39% Token: | 20|

Top 0th token. Logit: 17.22 Prob: 60.39% Token: | 20|
Top 1th token. Logit: 16.17 Prob: 21.14% Token: | 19|
Top 2th token. Logit: 14.34 Prob:  3.37% Token: | 21|


Ranks of the answer tokens: [(' 20', 0)]

# [ random word ] is [ digit ]

In [ ]:
example_prompt = "lamp is 1. red is 2. disa is 3. spoon is 4. time is"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'l', 'amp', ' is', ' 1', '.', ' red', ' is', ' 2', '.', ' dis', 'a', ' is', ' 3', '.', ' spoon', ' is', ' 4', '.', ' time', ' is']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 16.09 Prob: 57.95% Token: | 5|

Top 0th token. Logit: 16.09 Prob: 57.95% Token: | 5|
Top 1th token. Logit: 14.29 Prob:  9.66% Token: | 1|
Top 2th token. Logit: 13.42 Prob:  4.05% Token: | 6|
Top 3th token. Logit: 13.42 Prob:  4.04% Token: | 4|
Top 4th token. Logit: 12.56 Prob:  1.70% Token: | 10|
Top 5th token. Logit: 12.38 Prob:  1.43% Token: | 3|
Top 6th token. Logit: 12.36 Prob:  1.40% Token: | 2|
Top 7th token. Logit: 11.85 Prob:  0.84% Token: | 7|
Top 8th token. Logit: 11.75 Prob:  0.76% Token: | 8|
Top 9th token. Logit: 11.69 Prob:  0.72% Token: | 9|


Ranks of the answer tokens: [(' 5', 0)]

# [ number word ] is [ digit ]

In [ ]:
example_prompt = "One is 1. Two is 2. Three is 3. Four is"
example_answer = " 4"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'One', ' is', ' 1', '.', ' Two', ' is', ' 2', '.', ' Three', ' is', ' 3', '.', ' Four', ' is']
Tokenized answer: [' 4']


Performance on answer token:
Rank: 0        Logit: 16.52 Prob: 52.45% Token: | 4|

Top 0th token. Logit: 16.52 Prob: 52.45% Token: | 4|
Top 1th token. Logit: 16.12 Prob: 35.02% Token: | 5|
Top 2th token. Logit: 13.21 Prob:  1.92% Token: | 6|
Top 3th token. Logit: 12.48 Prob:  0.92% Token: | four|
Top 4th token. Logit: 12.22 Prob:  0.71% Token: | five|
Top 5th token. Logit: 11.99 Prob:  0.56% Token: | a|
Top 6th token. Logit: 11.76 Prob:  0.45% Token: | 1|
Top 7th token. Logit: 11.11 Prob:  0.23% Token: | not|
Top 8th token. Logit: 11.07 Prob:  0.22% Token: | the|
Top 9th token. Logit: 10.90 Prob:  0.19% Token: | 10|


Ranks of the answer tokens: [(' 4', 0)]

In [ ]:
example_prompt = "One is 1. Two is 2. Three is 3. Four is 4. Five is"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'One', ' is', ' 1', '.', ' Two', ' is', ' 2', '.', ' Three', ' is', ' 3', '.', ' Four', ' is', ' 4', '.', ' Five', ' is']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 16.34 Prob: 44.86% Token: | 5|

Top 0th token. Logit: 16.34 Prob: 44.86% Token: | 5|
Top 1th token. Logit: 16.33 Prob: 44.06% Token: | 6|
Top 2th token. Logit: 13.43 Prob:  2.43% Token: | 7|
Top 3th token. Logit: 12.06 Prob:  0.62% Token: | 1|
Top 4th token. Logit: 11.79 Prob:  0.47% Token: | a|
Top 5th token. Logit: 11.66 Prob:  0.41% Token: | five|
Top 6th token. Logit: 11.51 Prob:  0.36% Token: | 4|
Top 7th token. Logit: 11.43 Prob:  0.33% Token: | six|
Top 8th token. Logit: 10.96 Prob:  0.21% Token: | 10|
Top 9th token. Logit: 10.86 Prob:  0.19% Token: |
|


Ranks of the answer tokens: [(' 5', 0)]

In [ ]:
example_prompt = "One is 1. Two is 2. Three is 3. Four is 4. Five is 5. Six is 6. Seven is 7. Eight 8 is"
example_answer = " 8"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'One', ' is', ' 1', '.', ' Two', ' is', ' 2', '.', ' Three', ' is', ' 3', '.', ' Four', ' is', ' 4', '.', ' Five', ' is', ' 5', '.', ' Six', ' is', ' 6', '.', ' Seven', ' is', ' 7', '.', ' Eight', ' 8', ' is']
Tokenized answer: [' 8']


Performance on answer token:
Rank: 1        Logit: 13.06 Prob:  1.82% Token: | 8|

Top 0th token. Logit: 16.97 Prob: 90.43% Token: | 9|
Top 1th token. Logit: 13.06 Prob:  1.82% Token: | 8|
Top 2th token. Logit: 12.14 Prob:  0.72% Token: | 1|
Top 3th token. Logit: 11.67 Prob:  0.45% Token: | Nine|
Top 4th token. Logit: 11.60 Prob:  0.42% Token: | 10|
Top 5th token. Logit: 10.97 Prob:  0.22% Token: | nine|
Top 6th token. Logit: 10.96 Prob:  0.22% Token: |9|
Top 7th token. Logit: 10.76 Prob:  0.18% Token: |
|
Top 8th token. Logit: 10.57 Prob:  0.15% Token: | .|
Top 9th token. Logit: 10.51 Prob:  0.14% Token: | 0|


Ranks of the answer tokens: [(' 8', 1)]

## generated promps

In [ ]:
import inflect

def generate_prompts(N):
    p = inflect.engine()
    names = [p.number_to_words(i) for i in range(1, N+6)]  # Increase the range to prevent out of range error
    prompts = []
    answers = []
    corrupted_prompts = []

    for i in range(N):
        prompt = "{} is {}. {} is {}. {} is {}. {} is {}. {} is".format(
            names[i], i+1,
            names[i+1], i+2,
            names[i+2], i+3,
            names[i+3], i+4,
            names[i+4]
        )
        prompts.append(prompt)

        answers.append((str(i+5), str(i+4)))

        corrupted_prompt = "{} is {}. {} is {}. {} is {}. {} is {}. {} is".format(
            names[i], i+1,
            names[i+1], i+2,
            names[i+2], i+3,
            names[i+3], i+3,  # Here we repeat the number for corruption
            names[i+4]
        )
        corrupted_prompts.append(corrupted_prompt)

    return prompts, answers, corrupted_prompts

N = 16  # The number of prompts to be generated
prompts, answers, corrupted_prompts = generate_prompts(N)

# for i in range(N):
#     print('Prompt:', prompts[i])
#     print('Answer:', answers[i])
#     print('Corrupted Prompt:', corrupted_prompts[i])
#     print('----------')

In [ ]:
for i, prompt in enumerate(prompts):
    utils.test_prompt(prompt, answers[i][0], model, prepend_bos=True, top_k=3)

Tokenized prompt: ['<|endoftext|>', 'one', ' is', ' 1', '.', ' two', ' is', ' 2', '.', ' three', ' is', ' 3', '.', ' four', ' is', ' 4', '.', ' five', ' is']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 17.25 Prob: 71.74% Token: | 5|

Top 0th token. Logit: 17.25 Prob: 71.74% Token: | 5|
Top 1th token. Logit: 15.94 Prob: 19.25% Token: | 6|
Top 2th token. Logit: 12.80 Prob:  0.84% Token: | 4|


Ranks of the answer tokens: [(' 5', 0)]

Tokenized prompt: ['<|endoftext|>', 'two', ' is', ' 2', '.', ' three', ' is', ' 3', '.', ' four', ' is', ' 4', '.', ' five', ' is', ' 5', '.', ' six', ' is']
Tokenized answer: [' 6']


...

Ranks of the answer tokens: [(' 19', 0)]

Tokenized prompt: ['<|endoftext|>', 'six', 'teen', ' is', ' 16', '.', ' seventeen', ' is', ' 17', '.', ' eighteen', ' is', ' 18', '.', ' nineteen', ' is', ' 19', '.', ' twenty', ' is']
Tokenized answer: [' 20']


Performance on answer token:
Rank: 0        Logit: 17.01 Prob: 81.90% Token: | 20|

Top 0th token. Logit: 17.01 Prob: 81.90% Token: | 20|
Top 1th token. Logit: 14.08 Prob:  4.37% Token: | 21|
Top 2th token. Logit: 13.64 Prob:  2.83% Token: | twenty|


Ranks of the answer tokens: [(' 20', 0)]

In [ ]:
for i, prompt in enumerate(corrupted_prompts):
    utils.test_prompt(prompt, answers[i][1], model, prepend_bos=True, top_k=3)

Tokenized prompt: ['<|endoftext|>', 'one', ' is', ' 1', '.', ' two', ' is', ' 2', '.', ' three', ' is', ' 3', '.', ' four', ' is', ' 3', '.', ' five', ' is']
Tokenized answer: [' 4']


Performance on answer token:
Rank: 0        Logit: 17.51 Prob: 71.74% Token: | 4|

Top 0th token. Logit: 17.51 Prob: 71.74% Token: | 4|
Top 1th token. Logit: 15.86 Prob: 13.76% Token: | 5|
Top 2th token. Logit: 15.25 Prob:  7.50% Token: | 3|


Ranks of the answer tokens: [(' 4', 0)]

Tokenized prompt: ['<|endoftext|>', 'two', ' is', ' 2', '.', ' three', ' is', ' 3', '.', ' four', ' is', ' 4', '.', ' five', ' is', ' 4', '.', ' six', ' is']
Tokenized answer: [' 5']


...

Ranks of the answer tokens: [(' 18', 0)]

Tokenized prompt: ['<|endoftext|>', 'six', 'teen', ' is', ' 16', '.', ' seventeen', ' is', ' 17', '.', ' eighteen', ' is', ' 18', '.', ' nineteen', ' is', ' 18', '.', ' twenty', ' is']
Tokenized answer: [' 19']


Performance on answer token:
Rank: 0        Logit: 17.20 Prob: 68.17% Token: | 19|

Top 0th token. Logit: 17.20 Prob: 68.17% Token: | 19|
Top 1th token. Logit: 14.91 Prob:  6.94% Token: | 20|
Top 2th token. Logit: 14.69 Prob:  5.54% Token: | 18|


Ranks of the answer tokens: [(' 19', 0)]

## corrupt by flipping

In [ ]:
example_prompt = "One is 1. Two is 2. Three is 3. 4 is"
example_answer = " 4"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'One', ' is', ' 1', '.', ' Two', ' is', ' 2', '.', ' Three', ' is', ' 3', '.', ' 4', ' is']
Tokenized answer: [' 4']


Performance on answer token:
Rank: 0        Logit: 16.53 Prob: 55.59% Token: | 4|

Top 0th token. Logit: 16.53 Prob: 55.59% Token: | 4|
Top 1th token. Logit: 16.05 Prob: 34.54% Token: | 5|
Top 2th token. Logit: 12.47 Prob:  0.97% Token: | 6|
Top 3th token. Logit: 11.95 Prob:  0.57% Token: | 1|
Top 4th token. Logit: 11.88 Prob:  0.53% Token: | a|
Top 5th token. Logit: 11.30 Prob:  0.30% Token: | 2|
Top 6th token. Logit: 11.18 Prob:  0.27% Token: | the|
Top 7th token. Logit: 11.12 Prob:  0.25% Token: | not|
Top 8th token. Logit: 11.01 Prob:  0.22% Token: | four|
Top 9th token. Logit: 10.92 Prob:  0.20% Token: | 3|


Ranks of the answer tokens: [(' 4', 0)]

# [ name ] is [ number word ]

In [ ]:
import inflect

p = inflect.engine()

names = ["Adam", "Bob", "Claire", "Don", "Eve"]

prompts = ["{} is {}. {} is {}. {} is {}. {} is {}. {} is".format(names[0], p.number_to_words(i), names[1], p.number_to_words(i+1), names[2], p.number_to_words(i+2), names[3], p.number_to_words(i+3), names[4]) for i in range(1, 18)]
answers = [(p.number_to_words(i+4), p.number_to_words(i+3)) for i in range(1, 18)]
corrupted_prompts = ["{} is {}. {} is {}. {} is {}. {} is {}. {} is".format(names[0], p.number_to_words(i), names[1], p.number_to_words(i+1), names[2], p.number_to_words(i+2), names[3], p.number_to_words(i+2), names[4]) for i in range(1, 18)]


In [ ]:
for i, prompt in enumerate(prompts):
    utils.test_prompt(prompt, answers[i][0], model, prepend_bos=True, top_k=3)

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' one', '.', ' Bob', ' is', ' two', '.', ' Claire', ' is', ' three', '.', ' Don', ' is', ' four', '.', ' Eve', ' is']
Tokenized answer: [' five']


Performance on answer token:
Rank: 0        Logit: 18.36 Prob: 66.30% Token: | five|

Top 0th token. Logit: 18.36 Prob: 66.30% Token: | five|
Top 1th token. Logit: 15.63 Prob:  4.35% Token: | six|
Top 2th token. Logit: 15.57 Prob:  4.06% Token: | ten|


Ranks of the answer tokens: [(' five', 0)]

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' two', '.', ' Bob', ' is', ' three', '.', ' Claire', ' is', ' four', '.', ' Don', ' is', ' five', '.', ' Eve', ' is']
Tokenized answer: [' six']


...

Performance on answer token:
Rank: 0        Logit: 19.26 Prob: 64.27% Token: |-|

Top 0th token. Logit: 19.26 Prob: 64.27% Token: |-|
Top 1th token. Logit: 18.29 Prob: 24.25% Token: |.|
Top 2th token. Logit: 15.57 Prob:  1.61% Token: |,|


Performance on answer token:
Rank: 0        Logit: 20.34 Prob: 40.48% Token: |one|

Top 0th token. Logit: 20.34 Prob: 40.48% Token: |one|
Top 1th token. Logit: 18.95 Prob: 10.05% Token: |two|
Top 2th token. Logit: 18.94 Prob:  9.94% Token: |six|


Ranks of the answer tokens: [(' twenty', 0), ('-', 0), ('one', 0)]

# Test seqs to ensure pattern isn't occuring because "_ is _", but is detecting numbers

In [ ]:
example_prompt = "X is Y. X is Y. X is Y. X is Y. X is"
example_answer = " Y"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'X', ' is', ' Y', '.', ' X', ' is', ' Y', '.', ' X', ' is', ' Y', '.', ' X', ' is', ' Y', '.', ' X', ' is']
Tokenized answer: [' Y']


Performance on answer token:
Rank: 0        Logit: 19.68 Prob: 96.98% Token: | Y|

Top 0th token. Logit: 19.68 Prob: 96.98% Token: | Y|
Top 1th token. Logit: 15.25 Prob:  1.15% Token: | X|
Top 2th token. Logit: 14.16 Prob:  0.39% Token: | y|
Top 3th token. Logit: 13.31 Prob:  0.17% Token: | Z|
Top 4th token. Logit: 12.39 Prob:  0.07% Token: | the|
Top 5th token. Logit: 11.79 Prob:  0.04% Token: | a|
Top 6th token. Logit: 11.79 Prob:  0.04% Token: |Y|
Top 7th token. Logit: 11.74 Prob:  0.03% Token: | N|
Top 8th token. Logit: 11.67 Prob:  0.03% Token: | R|
Top 9th token. Logit: 11.58 Prob:  0.03% Token: |
|


Ranks of the answer tokens: [(' Y', 0)]

In [ ]:
example_prompt = "X is Y. X is Y. X is Y. X is Y. Y is"
example_answer = " X"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'X', ' is', ' Y', '.', ' X', ' is', ' Y', '.', ' X', ' is', ' Y', '.', ' X', ' is', ' Y', '.', ' Y', ' is']
Tokenized answer: [' X']


Performance on answer token:
Rank: 1        Logit: 15.83 Prob: 27.55% Token: | X|

Top 0th token. Logit: 16.44 Prob: 50.93% Token: | Y|
Top 1th token. Logit: 15.83 Prob: 27.55% Token: | X|
Top 2th token. Logit: 14.12 Prob:  5.00% Token: | Z|
Top 3th token. Logit: 12.79 Prob:  1.32% Token: | the|
Top 4th token. Logit: 12.78 Prob:  1.31% Token: | a|
Top 5th token. Logit: 11.84 Prob:  0.51% Token: | x|
Top 6th token. Logit: 11.67 Prob:  0.43% Token: | N|
Top 7th token. Logit: 11.67 Prob:  0.43% Token: |
|
Top 8th token. Logit: 11.53 Prob:  0.38% Token: | R|
Top 9th token. Logit: 11.52 Prob:  0.37% Token: | not|


Ranks of the answer tokens: [(' X', 1)]

In [ ]:
example_prompt = "Poland is Warsaw. France is Paris. China is Beijing. Spain is Madrid. Japan is"
example_answer = " Tokyo"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Pol', 'and', ' is', ' Warsaw', '.', ' France', ' is', ' Paris', '.', ' China', ' is', ' Beijing', '.', ' Spain', ' is', ' Madrid', '.', ' Japan', ' is']
Tokenized answer: [' Tokyo']


Performance on answer token:
Rank: 0        Logit: 19.62 Prob: 77.86% Token: | Tokyo|

Top 0th token. Logit: 19.62 Prob: 77.86% Token: | Tokyo|
Top 1th token. Logit: 16.09 Prob:  2.28% Token: | Japan|
Top 2th token. Logit: 15.84 Prob:  1.77% Token: | Osaka|
Top 3th token. Logit: 15.74 Prob:  1.61% Token: | Seoul|
Top 4th token. Logit: 15.33 Prob:  1.07% Token: | Yok|
Top 5th token. Logit: 15.21 Prob:  0.94% Token: | Kyoto|
Top 6th token. Logit: 15.04 Prob:  0.80% Token: | Beijing|
Top 7th token. Logit: 14.86 Prob:  0.66% Token: | Tai|
Top 8th token. Logit: 14.55 Prob:  0.49% Token: | the|
Top 9th token. Logit: 14.55 Prob:  0.49% Token: | Shanghai|


Ranks of the answer tokens: [(' Tokyo', 0)]

In [ ]:
example_prompt = "Poland is Warsaw. France is Paris. China is Beijing. Spain is Madrid. Spain is"
example_answer = " Madrid"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Pol', 'and', ' is', ' Warsaw', '.', ' France', ' is', ' Paris', '.', ' China', ' is', ' Beijing', '.', ' Spain', ' is', ' Madrid', '.', ' Spain', ' is']
Tokenized answer: [' Beijing']


Performance on answer token:
Rank: 10       Logit: 14.44 Prob:  2.13% Token: | Beijing|

Top 0th token. Logit: 16.17 Prob: 11.98% Token: | Madrid|
Top 1th token. Logit: 15.99 Prob:  9.99% Token: | Warsaw|
Top 2th token. Logit: 15.38 Prob:  5.40% Token: | Barcelona|
Top 3th token. Logit: 15.19 Prob:  4.49% Token: | the|
Top 4th token. Logit: 15.09 Prob:  4.06% Token: | Berlin|
Top 5th token. Logit: 14.97 Prob:  3.60% Token: | Rome|
Top 6th token. Logit: 14.79 Prob:  3.00% Token: | Paris|
Top 7th token. Logit: 14.67 Prob:  2.66% Token: | London|
Top 8th token. Logit: 14.64 Prob:  2.59% Token: | Budapest|
Top 9th token. Logit: 14.60 Prob:  2.48% Token: | Spain|


Ranks of the answer tokens: [(' Beijing', 10)]

In [ ]:
example_prompt = "A is A. A is A. A is A. A is A. A is"
example_answer = " A"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'A', ' is', ' A', '.', ' A', ' is', ' A', '.', ' A', ' is', ' A', '.', ' A', ' is', ' A', '.', ' A', ' is']
Tokenized answer: [' A']


Performance on answer token:
Rank: 0        Logit: 17.26 Prob: 91.65% Token: | A|

Top 0th token. Logit: 17.26 Prob: 91.65% Token: | A|
Top 1th token. Logit: 13.03 Prob:  1.33% Token: | B|
Top 2th token. Logit: 12.78 Prob:  1.03% Token: |
|
Top 3th token. Logit: 12.43 Prob:  0.73% Token: | a|
Top 4th token. Logit: 11.68 Prob:  0.35% Token: | An|
Top 5th token. Logit: 10.82 Prob:  0.15% Token: | The|
Top 6th token. Logit: 10.63 Prob:  0.12% Token: |A|
Top 7th token. Logit: 10.22 Prob:  0.08% Token: | Not|
Top 8th token. Logit: 10.10 Prob:  0.07% Token: | C|
Top 9th token. Logit:  9.98 Prob:  0.06% Token: | .|


Ranks of the answer tokens: [(' A', 0)]

In [ ]:
example_prompt = "A is A. A is A. A is A. A is A. B is"
example_answer = " A"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'A', ' is', ' A', '.', ' A', ' is', ' A', '.', ' A', ' is', ' A', '.', ' A', ' is', ' A', '.', ' B', ' is']
Tokenized answer: [' A']


Performance on answer token:
Rank: 1        Logit: 15.79 Prob: 16.99% Token: | A|

Top 0th token. Logit: 17.26 Prob: 74.18% Token: | B|
Top 1th token. Logit: 15.79 Prob: 16.99% Token: | A|
Top 2th token. Logit: 12.85 Prob:  0.90% Token: | C|
Top 3th token. Logit: 12.14 Prob:  0.44% Token: | a|
Top 4th token. Logit: 11.48 Prob:  0.23% Token: | D|
Top 5th token. Logit: 11.48 Prob:  0.23% Token: | b|
Top 6th token. Logit: 11.40 Prob:  0.21% Token: |
|
Top 7th token. Logit: 11.22 Prob:  0.18% Token: | The|
Top 8th token. Logit: 10.85 Prob:  0.12% Token: | Be|
Top 9th token. Logit: 10.83 Prob:  0.12% Token: | An|


Ranks of the answer tokens: [(' A', 1)]

In [ ]:
example_prompt = "lamp is lamp. lamp is lamp. lamp is lamp. lamp is lamp. house is"
example_answer = " house"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'l', 'amp', ' is', ' lamp', '.', ' lamp', ' is', ' lamp', '.', ' lamp', ' is', ' lamp', '.', ' lamp', ' is', ' lamp', '.', ' house', ' is']
Tokenized answer: [' house']


Performance on answer token:
Rank: 0        Logit: 16.69 Prob: 64.80% Token: | house|

Top 0th token. Logit: 16.69 Prob: 64.80% Token: | house|
Top 1th token. Logit: 14.07 Prob:  4.74% Token: | lamp|
Top 2th token. Logit: 13.64 Prob:  3.07% Token: | a|
Top 3th token. Logit: 13.49 Prob:  2.64% Token: | home|
Top 4th token. Logit: 12.80 Prob:  1.32% Token: | light|
Top 5th token. Logit: 11.86 Prob:  0.52% Token: | kitchen|
Top 6th token. Logit: 11.69 Prob:  0.44% Token: | the|
Top 7th token. Logit: 11.63 Prob:  0.41% Token: | furniture|
Top 8th token. Logit: 11.54 Prob:  0.38% Token: | not|
Top 9th token. Logit: 11.33 Prob:  0.30% Token: | room|


Ranks of the answer tokens: [(' house', 0)]

In [ ]:
example_prompt = "Adam is Adam. Adam is Adam. Adam is Adam. house is house. house is"
example_answer = " house"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Adam', ' is', ' Adam', '.', ' Adam', ' is', ' Adam', '.', ' Adam', ' is', ' Adam', '.', ' house', ' is', ' house', '.', ' house', ' is']
Tokenized answer: [' house']


Performance on answer token:
Rank: 0        Logit: 17.48 Prob: 89.29% Token: | house|

Top 0th token. Logit: 17.48 Prob: 89.29% Token: | house|
Top 1th token. Logit: 13.07 Prob:  1.09% Token: | home|
Top 2th token. Logit: 12.22 Prob:  0.47% Token: | the|
Top 3th token. Logit: 11.93 Prob:  0.35% Token: | a|
Top 4th token. Logit: 11.93 Prob:  0.35% Token: | family|
Top 5th token. Logit: 11.17 Prob:  0.16% Token: | room|
Top 6th token. Logit: 11.06 Prob:  0.15% Token: | place|
Top 7th token. Logit: 10.96 Prob:  0.13% Token: | House|
Top 8th token. Logit: 10.84 Prob:  0.12% Token: | kitchen|
Top 9th token. Logit: 10.72 Prob:  0.10% Token: | not|


Ranks of the answer tokens: [(' house', 0)]

In [ ]:
example_prompt = "Tall is short. Big is small. Heavy is light. Far is close. Long is"
example_answer = " thin"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'T', 'all', ' is', ' short', '.', ' Big', ' is', ' small', '.', ' Heavy', ' is', ' light', '.', ' Far', ' is', ' close', '.', ' Long', ' is']
Tokenized answer: [' thin']


Performance on answer token:
Rank: 17       Logit: 12.95 Prob:  0.30% Token: | thin|

Top 0th token. Logit: 17.81 Prob: 38.43% Token: | long|
Top 1th token. Logit: 17.37 Prob: 24.74% Token: | far|
Top 2th token. Logit: 16.38 Prob:  9.19% Token: | short|
Top 3th token. Logit: 14.93 Prob:  2.14% Token: | wide|
Top 4th token. Logit: 14.39 Prob:  1.25% Token: | deep|
Top 5th token. Logit: 14.20 Prob:  1.04% Token: | close|
Top 6th token. Logit: 14.08 Prob:  0.92% Token: | distant|
Top 7th token. Logit: 14.06 Prob:  0.90% Token: | near|
Top 8th token. Logit: 13.97 Prob:  0.82% Token: | the|
Top 9th token. Logit: 13.67 Prob:  0.61% Token: | very|


Ranks of the answer tokens: [(' thin', 17)]

In [ ]:
# https://bobbyhadz.com/blog/python-generate-random-word
import random
import requests

def get_list_of_words():
    response = requests.get(
        'https://www.mit.edu/~ecprice/wordlist.10000',
        timeout=10
    )

    string_of_words = response.content.decode('utf-8')

    list_of_words = string_of_words.splitlines()

    return list_of_words

word_list = get_list_of_words()

In [ ]:
def generate_sentences(n, word_list):
    sentences = []
    word_tuples = []
    corr_sentences = []
    # for _ in range(n):
    while len(sentences) < n:
        first_word = random.choice(word_list)
        second_word = random.choice(word_list)
        sentence = f"{first_word} is {first_word}. {first_word} is {first_word}. {first_word} is {first_word}. {first_word} is {first_word}. {first_word} is"
        corr_sentence = f"{first_word} is {first_word}. {first_word} is {first_word}. {first_word} is {first_word}. {first_word} is {first_word}. {second_word} is"
        # tokenzier seems to work differently when tokens used with other tokens?
        if len(model.to_str_tokens(sentence)) == 19 and len(model.to_str_tokens(corr_sentence)) == 19:
            sentences.append(sentence)
            word_tuples.append((first_word, second_word))
            corr_sentences.append(corr_sentence)
    return sentences, word_tuples, corr_sentences

num_sentences = 15

prompts, answers, corrupted_prompts = generate_sentences(num_sentences, word_list)

In [ ]:
for i, prompt in enumerate(corrupted_prompts):
    utils.test_prompt(prompt, answers[i][1], model, prepend_bos=True, top_k=3)

Tokenized prompt: ['<|endoftext|>', 'da', ' is', ' da', '.', ' da', ' is', ' da', '.', ' da', ' is', ' da', '.', ' da', ' is', ' da', '.', ' behind', ' is']
Tokenized answer: [' behind']


Performance on answer token:
Rank: 0        Logit: 15.98 Prob: 57.94% Token: | behind|

Top 0th token. Logit: 15.98 Prob: 57.94% Token: | behind|
Top 1th token. Logit: 14.15 Prob:  9.28% Token: | da|
Top 2th token. Logit: 12.71 Prob:  2.21% Token: | a|


Ranks of the answer tokens: [(' behind', 0)]

Tokenized prompt: ['<|endoftext|>', 'particularly', ' is', ' particularly', '.', ' particularly', ' is', ' particularly', '.', ' particularly', ' is', ' particularly', '.', ' particularly', ' is', ' particularly', '.', ' chicken', ' is']
Tokenized answer: [' chicken']


...

Ranks of the answer tokens: [(' pure', 0)]

Tokenized prompt: ['<|endoftext|>', 'lower', ' is', ' lower', '.', ' lower', ' is', ' lower', '.', ' lower', ' is', ' lower', '.', ' lower', ' is', ' lower', '.', ' dreams', ' is']
Tokenized answer: [' dreams']


Performance on answer token:
Rank: 1        Logit: 12.14 Prob:  4.22% Token: | dreams|

Top 0th token. Logit: 14.47 Prob: 43.57% Token: | lower|
Top 1th token. Logit: 12.14 Prob:  4.22% Token: | dreams|
Top 2th token. Logit: 12.13 Prob:  4.20% Token: | higher|


Ranks of the answer tokens: [(' dreams', 1)]

# Fibonacci

In [ ]:
example_prompt = "0, 1, 1, 2, 3, 5,"
example_answer = " 8"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ',', ' 1', ',', ' 1', ',', ' 2', ',', ' 3', ',', ' 5', ',']
Tokenized answer: [' 8']


Performance on answer token:
Rank: 3        Logit: 16.12 Prob:  8.50% Token: | 8|

Top 0th token. Logit: 17.46 Prob: 32.54% Token: | 6|
Top 1th token. Logit: 17.32 Prob: 28.18% Token: | 7|
Top 2th token. Logit: 16.31 Prob: 10.30% Token: | 10|
Top 3th token. Logit: 16.12 Prob:  8.50% Token: | 8|
Top 4th token. Logit: 15.59 Prob:  4.99% Token: | 9|
Top 5th token. Logit: 14.98 Prob:  2.73% Token: | 11|
Top 6th token. Logit: 14.63 Prob:  1.91% Token: | 5|
Top 7th token. Logit: 14.53 Prob:  1.72% Token: | 12|
Top 8th token. Logit: 14.33 Prob:  1.41% Token: | 1|
Top 9th token. Logit: 13.49 Prob:  0.61% Token: | 2|


Ranks of the answer tokens: [(' 8', 3)]

In [ ]:
example_prompt = "0 1 1 2 3 5"
example_answer = " 8"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ' 1', ' 1', ' 2', ' 3', ' 5']
Tokenized answer: [' 8']


Performance on answer token:
Rank: 3        Logit: 14.84 Prob:  2.82% Token: | 8|

Top 0th token. Logit: 18.06 Prob: 70.52% Token: | 6|
Top 1th token. Logit: 15.19 Prob:  4.01% Token: |
|
Top 2th token. Logit: 15.06 Prob:  3.53% Token: | 0|
Top 3th token. Logit: 14.84 Prob:  2.82% Token: | 8|
Top 4th token. Logit: 14.80 Prob:  2.71% Token: | 10|
Top 5th token. Logit: 14.66 Prob:  2.35% Token: | 7|
Top 6th token. Logit: 14.49 Prob:  1.99% Token: | 5|
Top 7th token. Logit: 14.22 Prob:  1.52% Token: | 4|
Top 8th token. Logit: 13.74 Prob:  0.94% Token: | 9|
Top 9th token. Logit: 13.38 Prob:  0.66% Token: | 1|


Ranks of the answer tokens: [(' 8', 3)]

In [ ]:
example_prompt = "0 1 1 2 3 5 8 13 21"
example_answer = " 34"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ' 1', ' 1', ' 2', ' 3', ' 5', ' 8', ' 13', ' 21']
Tokenized answer: [' 34']


Performance on answer token:
Rank: 17       Logit: 16.04 Prob:  1.55% Token: | 34|

Top 0th token. Logit: 17.60 Prob:  7.37% Token: | 22|
Top 1th token. Logit: 17.58 Prob:  7.26% Token: | 30|
Top 2th token. Logit: 17.43 Prob:  6.25% Token: | 25|
Top 3th token. Logit: 17.39 Prob:  5.97% Token: | 24|
Top 4th token. Logit: 17.31 Prob:  5.52% Token: |
|
Top 5th token. Logit: 17.12 Prob:  4.58% Token: | 32|
Top 6th token. Logit: 17.09 Prob:  4.44% Token: | 23|
Top 7th token. Logit: 16.99 Prob:  4.00% Token: | 20|
Top 8th token. Logit: 16.87 Prob:  3.56% Token: | 29|
Top 9th token. Logit: 16.72 Prob:  3.07% Token: | 21|


Ranks of the answer tokens: [(' 34', 17)]

# only have last K elems be increasing seq

In [ ]:
example_prompt = "1 3 5 1 2 3"
example_answer = " 4"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 3', ' 5', ' 1', ' 2', ' 3']
Tokenized answer: [' 4']


Performance on answer token:
Rank: 0        Logit: 15.54 Prob: 30.20% Token: | 4|

Top 0th token. Logit: 15.54 Prob: 30.20% Token: | 4|
Top 1th token. Logit: 15.49 Prob: 28.70% Token: | 5|
Top 2th token. Logit: 13.99 Prob:  6.40% Token: | 6|
Top 3th token. Logit: 13.87 Prob:  5.71% Token: | 2|
Top 4th token. Logit: 13.77 Prob:  5.15% Token: | 3|
Top 5th token. Logit: 13.75 Prob:  5.03% Token: | 1|
Top 6th token. Logit: 13.41 Prob:  3.59% Token: |
|
Top 7th token. Logit: 12.74 Prob:  1.83% Token: | 0|
Top 8th token. Logit: 11.74 Prob:  0.68% Token: | 8|
Top 9th token. Logit: 11.54 Prob:  0.55% Token: | 7|


Ranks of the answer tokens: [(' 4', 0)]

In [ ]:
example_prompt = "1 3 3 1 2 3"
example_answer = " 4"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 3', ' 3', ' 1', ' 2', ' 3']
Tokenized answer: [' 4']


Performance on answer token:
Rank: 3        Logit: 13.71 Prob: 10.96% Token: | 4|

Top 0th token. Logit: 14.68 Prob: 29.10% Token: | 3|
Top 1th token. Logit: 14.09 Prob: 16.07% Token: | 2|
Top 2th token. Logit: 14.06 Prob: 15.66% Token: | 1|
Top 3th token. Logit: 13.71 Prob: 10.96% Token: | 4|
Top 4th token. Logit: 12.92 Prob:  4.97% Token: |
|
Top 5th token. Logit: 12.55 Prob:  3.44% Token: | 0|
Top 6th token. Logit: 11.76 Prob:  1.56% Token: | 5|
Top 7th token. Logit: 11.09 Prob:  0.80% Token: | 6|
Top 8th token. Logit: 10.88 Prob:  0.65% Token: |.|
Top 9th token. Logit: 10.17 Prob:  0.32% Token: | -|


Ranks of the answer tokens: [(' 4', 3)]

Changing 5 to 3 corrupts this. So it's not just looking a few back. See how this is recovered. See what looks at 3, what looks at 2, etc.

## len 4

In [ ]:
example_prompt = "1 2 3 4"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 3', ' 4']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 18.76 Prob: 96.18% Token: | 5|

Top 0th token. Logit: 18.76 Prob: 96.18% Token: | 5|
Top 1th token. Logit: 13.27 Prob:  0.40% Token: | Next|
Top 2th token. Logit: 13.01 Prob:  0.30% Token: |
|
Top 3th token. Logit: 12.87 Prob:  0.27% Token: | >|
Top 4th token. Logit: 12.04 Prob:  0.12% Token: | 4|
Top 5th token. Logit: 11.88 Prob:  0.10% Token: | 50|
Top 6th token. Logit: 11.83 Prob:  0.09% Token: | 6|
Top 7th token. Logit: 11.71 Prob:  0.08% Token: | <|
Top 8th token. Logit: 11.64 Prob:  0.08% Token: | $|
Top 9th token. Logit: 11.63 Prob:  0.08% Token: | 1|


Ranks of the answer tokens: [(' 5', 0)]

### corr n-2

In [ ]:
example_prompt = "1 3 3 4"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 3', ' 3', ' 4']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 18.59 Prob: 95.66% Token: | 5|

Top 0th token. Logit: 18.59 Prob: 95.66% Token: | 5|
Top 1th token. Logit: 13.55 Prob:  0.62% Token: | 4|
Top 2th token. Logit: 13.19 Prob:  0.43% Token: |
|
Top 3th token. Logit: 12.47 Prob:  0.21% Token: | Next|
Top 4th token. Logit: 12.46 Prob:  0.21% Token: | 6|
Top 5th token. Logit: 11.90 Prob:  0.12% Token: | 1|
Top 6th token. Logit: 11.88 Prob:  0.12% Token: | 3|
Top 7th token. Logit: 11.74 Prob:  0.10% Token: | -|
Top 8th token. Logit: 11.66 Prob:  0.09% Token: | +|
Top 9th token. Logit: 11.60 Prob:  0.09% Token: | >|


Ranks of the answer tokens: [(' 5', 0)]

In [ ]:
example_prompt = "1 1 3 4"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 1', ' 3', ' 4']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 18.54 Prob: 95.63% Token: | 5|

Top 0th token. Logit: 18.54 Prob: 95.63% Token: | 5|
Top 1th token. Logit: 13.16 Prob:  0.44% Token: |
|
Top 2th token. Logit: 12.86 Prob:  0.33% Token: | 6|
Top 3th token. Logit: 12.48 Prob:  0.22% Token: | 4|
Top 4th token. Logit: 12.28 Prob:  0.18% Token: | 1|
Top 5th token. Logit: 12.27 Prob:  0.18% Token: | 7|
Top 6th token. Logit: 12.08 Prob:  0.15% Token: | Next|
Top 7th token. Logit: 11.90 Prob:  0.13% Token: | -|
Top 8th token. Logit: 11.86 Prob:  0.12% Token: | 3|
Top 9th token. Logit: 11.82 Prob:  0.12% Token: | +|


Ranks of the answer tokens: [(' 5', 0)]

In [ ]:
example_prompt = "1 blue 3 4"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' blue', ' 3', ' 4']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 14.74 Prob: 57.16% Token: | 5|

Top 0th token. Logit: 14.74 Prob: 57.16% Token: | 5|
Top 1th token. Logit: 12.42 Prob:  5.65% Token: | 3|
Top 2th token. Logit: 12.07 Prob:  3.96% Token: | 1|
Top 3th token. Logit: 11.96 Prob:  3.54% Token: | 4|
Top 4th token. Logit: 11.29 Prob:  1.82% Token: |
|
Top 5th token. Logit: 11.24 Prob:  1.74% Token: | 6|
Top 6th token. Logit: 11.20 Prob:  1.66% Token: | 0|
Top 7th token. Logit: 11.12 Prob:  1.54% Token: | 7|
Top 8th token. Logit: 10.89 Prob:  1.23% Token: | red|
Top 9th token. Logit: 10.74 Prob:  1.05% Token: | 2|


Ranks of the answer tokens: [(' 5', 0)]

n-2 DOES affect it. But not by much. Not so much that we can use actv patching to find strong differentiating signals

### corr n-1

In [ ]:
example_prompt = "1 1 5 4"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 1', ' 5', ' 4']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 16.08 Prob: 37.83% Token: | 5|

Top 0th token. Logit: 16.08 Prob: 37.83% Token: | 5|
Top 1th token. Logit: 15.10 Prob: 14.27% Token: | 3|
Top 2th token. Logit: 14.45 Prob:  7.42% Token: | 1|
Top 3th token. Logit: 14.33 Prob:  6.58% Token: | 4|
Top 4th token. Logit: 13.96 Prob:  4.56% Token: | 6|
Top 5th token. Logit: 13.96 Prob:  4.55% Token: | 2|
Top 6th token. Logit: 13.74 Prob:  3.64% Token: |
|
Top 7th token. Logit: 13.58 Prob:  3.10% Token: | 7|
Top 8th token. Logit: 13.29 Prob:  2.33% Token: | 0|
Top 9th token. Logit: 12.37 Prob:  0.93% Token: | 10|


Ranks of the answer tokens: [(' 5', 0)]

In [ ]:
example_prompt = "1 1 blue 4"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 1', ' blue', ' 4']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 6        Logit: 10.62 Prob:  2.61% Token: | 5|

Top 0th token. Logit: 11.89 Prob:  9.25% Token: |.|
Top 1th token. Logit: 11.74 Prob:  7.97% Token: | 1|
Top 2th token. Logit: 11.19 Prob:  4.60% Token: | 3|
Top 3th token. Logit: 11.17 Prob:  4.51% Token: | 2|
Top 4th token. Logit: 11.11 Prob:  4.24% Token: | 0|
Top 5th token. Logit: 10.92 Prob:  3.51% Token: | 4|
Top 6th token. Logit: 10.62 Prob:  2.61% Token: | 5|
Top 7th token. Logit: 10.56 Prob:  2.45% Token: |,|
Top 8th token. Logit: 10.52 Prob:  2.36% Token: |
|
Top 9th token. Logit: 10.05 Prob:  1.47% Token: |-|


Ranks of the answer tokens: [(' 5', 6)]

In the case of len 4, disrupting the n-2 previous (2) doesn't do much! It only depends on the last one (3, n-1)

In [ ]:
example_prompt = "1 1 1 4"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 1', ' 1', ' 4']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 4        Logit: 12.18 Prob:  5.43% Token: | 5|

Top 0th token. Logit: 13.22 Prob: 15.37% Token: | 1|
Top 1th token. Logit: 12.42 Prob:  6.93% Token: | 2|
Top 2th token. Logit: 12.39 Prob:  6.72% Token: | 3|
Top 3th token. Logit: 12.35 Prob:  6.44% Token: | 4|
Top 4th token. Logit: 12.18 Prob:  5.43% Token: | 5|
Top 5th token. Logit: 12.15 Prob:  5.27% Token: |.|
Top 6th token. Logit: 11.99 Prob:  4.49% Token: |
|
Top 7th token. Logit: 11.81 Prob:  3.76% Token: | 0|
Top 8th token. Logit: 11.34 Prob:  2.35% Token: | 6|
Top 9th token. Logit: 10.82 Prob:  1.40% Token: | 8|


Ranks of the answer tokens: [(' 5', 4)]

In [ ]:
example_prompt = "blue 2 3 4"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'blue', ' 2', ' 3', ' 4']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 16.01 Prob: 85.93% Token: | 5|

Top 0th token. Logit: 16.01 Prob: 85.93% Token: | 5|
Top 1th token. Logit: 11.79 Prob:  1.27% Token: |
|
Top 2th token. Logit: 11.37 Prob:  0.83% Token: | 6|
Top 3th token. Logit: 11.27 Prob:  0.75% Token: | 4|
Top 4th token. Logit: 10.94 Prob:  0.54% Token: | 3|
Top 5th token. Logit: 10.84 Prob:  0.49% Token: | 1|
Top 6th token. Logit: 10.62 Prob:  0.39% Token: | 7|
Top 7th token. Logit: 10.42 Prob:  0.32% Token: | 0|
Top 8th token. Logit:  9.72 Prob:  0.16% Token: | +|
Top 9th token. Logit:  9.65 Prob:  0.15% Token: | -|


Ranks of the answer tokens: [(' 5', 0)]

## len 5

corr ways: change into:
- a diff num
- same num
- change to a rand word (blue)

### n-1

In [ ]:
example_prompt = "1 2 3 5 5"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 3', ' 5', ' 5']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 15.40 Prob: 49.94% Token: | 6|

Top 0th token. Logit: 15.40 Prob: 49.94% Token: | 6|
Top 1th token. Logit: 13.59 Prob:  8.17% Token: | 5|
Top 2th token. Logit: 13.09 Prob:  4.96% Token: |
|
Top 3th token. Logit: 12.63 Prob:  3.13% Token: | 4|
Top 4th token. Logit: 12.11 Prob:  1.86% Token: | 7|
Top 5th token. Logit: 11.90 Prob:  1.51% Token: | 8|
Top 6th token. Logit: 11.88 Prob:  1.48% Token: | 10|
Top 7th token. Logit: 11.73 Prob:  1.28% Token: | 1|
Top 8th token. Logit: 11.56 Prob:  1.08% Token: | 0|
Top 9th token. Logit: 11.40 Prob:  0.92% Token: | 9|


Ranks of the answer tokens: [(' 6', 0)]

In [ ]:
example_prompt = "1 2 3 3 5"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 3', ' 3', ' 5']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 16.47 Prob: 69.80% Token: | 6|

Top 0th token. Logit: 16.47 Prob: 69.80% Token: | 6|
Top 1th token. Logit: 13.99 Prob:  5.83% Token: | 4|
Top 2th token. Logit: 13.62 Prob:  4.04% Token: | 5|
Top 3th token. Logit: 12.73 Prob:  1.64% Token: |
|
Top 4th token. Logit: 12.51 Prob:  1.32% Token: | 8|
Top 5th token. Logit: 12.00 Prob:  0.79% Token: | >|
Top 6th token. Logit: 11.87 Prob:  0.70% Token: | 10|
Top 7th token. Logit: 11.79 Prob:  0.65% Token: | 7|
Top 8th token. Logit: 11.66 Prob:  0.57% Token: | 11|
Top 9th token. Logit: 11.61 Prob:  0.54% Token: | 26|


Ranks of the answer tokens: [(' 6', 0)]

In [ ]:
example_prompt = "1 2 3 1 5"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 3', ' 1', ' 5']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 2        Logit: 12.80 Prob:  8.24% Token: | 6|

Top 0th token. Logit: 13.20 Prob: 12.32% Token: | 5|
Top 1th token. Logit: 12.93 Prob:  9.39% Token: | 4|
Top 2th token. Logit: 12.80 Prob:  8.24% Token: | 6|
Top 3th token. Logit: 12.48 Prob:  5.98% Token: | 2|
Top 4th token. Logit: 12.47 Prob:  5.96% Token: | 3|
Top 5th token. Logit: 12.43 Prob:  5.69% Token: | 1|
Top 6th token. Logit: 11.98 Prob:  3.65% Token: |
|
Top 7th token. Logit: 11.58 Prob:  2.45% Token: | 0|
Top 8th token. Logit: 11.25 Prob:  1.75% Token: | 8|
Top 9th token. Logit: 11.15 Prob:  1.58% Token: | 10|


Ranks of the answer tokens: [(' 6', 2)]

It matters a lot what numbers is chosen as the corruption.

### n-2

In [ ]:
example_prompt = "1 2 5 4 5"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 5', ' 4', ' 5']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 14.96 Prob: 25.85% Token: | 6|

Top 0th token. Logit: 14.96 Prob: 25.85% Token: | 6|
Top 1th token. Logit: 14.58 Prob: 17.72% Token: | 5|
Top 2th token. Logit: 13.31 Prob:  4.96% Token: |
|
Top 3th token. Logit: 13.18 Prob:  4.37% Token: | 4|
Top 4th token. Logit: 12.92 Prob:  3.36% Token: | 3|
Top 5th token. Logit: 12.64 Prob:  2.54% Token: | 1|
Top 6th token. Logit: 12.57 Prob:  2.38% Token: | 0|
Top 7th token. Logit: 12.48 Prob:  2.16% Token: | 7|
Top 8th token. Logit: 12.26 Prob:  1.73% Token: | 9|
Top 9th token. Logit: 12.22 Prob:  1.66% Token: | 2|


Ranks of the answer tokens: [(' 6', 0)]

In [ ]:
example_prompt = "1 2 1 4 5"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 1', ' 4', ' 5']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 16.99 Prob: 83.38% Token: | 6|

Top 0th token. Logit: 16.99 Prob: 83.38% Token: | 6|
Top 1th token. Logit: 12.86 Prob:  1.35% Token: | 5|
Top 2th token. Logit: 12.79 Prob:  1.25% Token: |
|
Top 3th token. Logit: 12.67 Prob:  1.11% Token: | 7|
Top 4th token. Logit: 12.39 Prob:  0.84% Token: | 8|
Top 5th token. Logit: 12.31 Prob:  0.78% Token: | 4|
Top 6th token. Logit: 11.96 Prob:  0.55% Token: | 0|
Top 7th token. Logit: 11.96 Prob:  0.55% Token: | 1|
Top 8th token. Logit: 11.95 Prob:  0.54% Token: | +|
Top 9th token. Logit: 11.65 Prob:  0.40% Token: | 9|


Ranks of the answer tokens: [(' 6', 0)]

In [ ]:
example_prompt = "1 2 blue 4 5"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' blue', ' 4', ' 5']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 13.13 Prob: 22.82% Token: | 6|

Top 0th token. Logit: 13.13 Prob: 22.82% Token: | 6|
Top 1th token. Logit: 11.96 Prob:  7.07% Token: | 7|
Top 2th token. Logit: 11.83 Prob:  6.25% Token: | 5|
Top 3th token. Logit: 11.68 Prob:  5.36% Token: | red|
Top 4th token. Logit: 11.02 Prob:  2.76% Token: | blue|
Top 5th token. Logit: 10.86 Prob:  2.37% Token: |
|
Top 6th token. Logit: 10.69 Prob:  1.99% Token: | 1|
Top 7th token. Logit: 10.64 Prob:  1.90% Token: | 3|
Top 8th token. Logit: 10.47 Prob:  1.60% Token: | 4|
Top 9th token. Logit: 10.35 Prob:  1.42% Token: | 0|


Ranks of the answer tokens: [(' 6', 0)]

In [ ]:
example_prompt = "blue blue blue 4 5"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'blue', ' blue', ' blue', ' 4', ' 5']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 12.48 Prob: 18.79% Token: | 6|

Top 0th token. Logit: 12.48 Prob: 18.79% Token: | 6|
Top 1th token. Logit: 11.84 Prob:  9.90% Token: | 5|
Top 2th token. Logit: 11.40 Prob:  6.38% Token: | 4|
Top 3th token. Logit: 11.15 Prob:  4.96% Token: |
|
Top 4th token. Logit: 10.90 Prob:  3.89% Token: | 7|
Top 5th token. Logit: 10.69 Prob:  3.15% Token: | 0|
Top 6th token. Logit: 10.48 Prob:  2.55% Token: | blue|
Top 7th token. Logit: 10.40 Prob:  2.36% Token: | 1|
Top 8th token. Logit: 10.34 Prob:  2.21% Token: | 3|
Top 9th token. Logit: 10.11 Prob:  1.76% Token: | 8|


Ranks of the answer tokens: [(' 6', 0)]

len 5 also just depends on n-1 mostly

Is there a point where the repeats overtakes the sequence?

### n-3

In [ ]:
example_prompt = "1 blue 3 4 5"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' blue', ' 3', ' 4', ' 5']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 15.98 Prob: 78.05% Token: | 6|

Top 0th token. Logit: 15.98 Prob: 78.05% Token: | 6|
Top 1th token. Logit: 12.49 Prob:  2.37% Token: | 5|
Top 2th token. Logit: 12.42 Prob:  2.22% Token: |
|
Top 3th token. Logit: 12.11 Prob:  1.62% Token: | 7|
Top 4th token. Logit: 11.80 Prob:  1.18% Token: | 4|
Top 5th token. Logit: 11.64 Prob:  1.01% Token: | 0|
Top 6th token. Logit: 11.39 Prob:  0.79% Token: | 1|
Top 7th token. Logit: 11.30 Prob:  0.72% Token: | 8|
Top 8th token. Logit: 10.80 Prob:  0.44% Token: | red|
Top 9th token. Logit: 10.63 Prob:  0.37% Token: | +|


Ranks of the answer tokens: [(' 6', 0)]

In [ ]:
example_prompt = "1 null 3 4 5"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' null', ' 3', ' 4', ' 5']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 16.30 Prob: 78.65% Token: | 6|

Top 0th token. Logit: 16.30 Prob: 78.65% Token: | 6|
Top 1th token. Logit: 12.67 Prob:  2.08% Token: | 7|
Top 2th token. Logit: 12.59 Prob:  1.93% Token: |
|
Top 3th token. Logit: 12.15 Prob:  1.24% Token: | 5|
Top 4th token. Logit: 12.10 Prob:  1.18% Token: | 0|
Top 5th token. Logit: 11.88 Prob:  0.95% Token: | 8|
Top 6th token. Logit: 11.40 Prob:  0.58% Token: | 1|
Top 7th token. Logit: 11.24 Prob:  0.50% Token: | 11|
Top 8th token. Logit: 11.23 Prob:  0.50% Token: | 4|
Top 9th token. Logit: 11.08 Prob:  0.43% Token: | 9|


Ranks of the answer tokens: [(' 6', 0)]

In [ ]:
example_prompt = "3 4 5"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '3', ' 4', ' 5']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 15.55 Prob: 56.90% Token: | 6|

Top 0th token. Logit: 15.55 Prob: 56.90% Token: | 6|
Top 1th token. Logit: 13.06 Prob:  4.70% Token: |
|
Top 2th token. Logit: 13.03 Prob:  4.56% Token: | 5|
Top 3th token. Logit: 12.59 Prob:  2.95% Token: | 4|
Top 4th token. Logit: 12.42 Prob:  2.49% Token: | 8|
Top 5th token. Logit: 12.37 Prob:  2.36% Token: | 7|
Top 6th token. Logit: 11.53 Prob:  1.02% Token: | 0|
Top 7th token. Logit: 11.43 Prob:  0.93% Token: | 10|
Top 8th token. Logit: 11.40 Prob:  0.89% Token: | +|
Top 9th token. Logit: 11.39 Prob:  0.89% Token: | 9|


Ranks of the answer tokens: [(' 6', 0)]

In [ ]:
example_prompt = "blue blue 3 4 5"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'blue', ' blue', ' 3', ' 4', ' 5']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 15.23 Prob: 63.80% Token: | 6|

Top 0th token. Logit: 15.23 Prob: 63.80% Token: | 6|
Top 1th token. Logit: 12.44 Prob:  3.93% Token: | 5|
Top 2th token. Logit: 12.15 Prob:  2.94% Token: |
|
Top 3th token. Logit: 12.14 Prob:  2.90% Token: | 7|
Top 4th token. Logit: 12.06 Prob:  2.69% Token: | 4|
Top 5th token. Logit: 11.51 Prob:  1.54% Token: | 8|
Top 6th token. Logit: 11.39 Prob:  1.37% Token: | blue|
Top 7th token. Logit: 11.32 Prob:  1.28% Token: | 0|
Top 8th token. Logit: 10.97 Prob:  0.90% Token: | red|
Top 9th token. Logit: 10.90 Prob:  0.84% Token: | 1|


Ranks of the answer tokens: [(' 6', 0)]

# corrupt at the end

In [ ]:
example_prompt = "1 2 3 4 blue"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 3', ' 4', ' blue']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 12       Logit:  8.15 Prob:  0.56% Token: | 5|

Top 0th token. Logit: 10.71 Prob:  7.19% Token: |_|
Top 1th token. Logit:  9.47 Prob:  2.08% Token: |berry|
Top 2th token. Logit:  9.21 Prob:  1.61% Token: |-|
Top 3th token. Logit:  9.06 Prob:  1.39% Token: | 1|
Top 4th token. Logit:  8.88 Prob:  1.16% Token: |bird|
Top 5th token. Logit:  8.51 Prob:  0.80% Token: | (|
Top 6th token. Logit:  8.50 Prob:  0.79% Token: | 2|
Top 7th token. Logit:  8.40 Prob:  0.72% Token: |1|
Top 8th token. Logit:  8.35 Prob:  0.68% Token: | 0|
Top 9th token. Logit:  8.29 Prob:  0.64% Token: |2|


Ranks of the answer tokens: [(' 5', 12)]

# add 2

In [ ]:
example_prompt = "2 4 6 8 10 12"  #6 is minimal
example_answer = " 14"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 6', ' 8', ' 10', ' 12']
Tokenized answer: [' 14']


Performance on answer token:
Rank: 0        Logit: 18.26 Prob: 35.59% Token: | 14|

Top 0th token. Logit: 18.26 Prob: 35.59% Token: | 14|
Top 1th token. Logit: 17.97 Prob: 26.59% Token: | 13|
Top 2th token. Logit: 17.67 Prob: 19.57% Token: | 15|
Top 3th token. Logit: 15.95 Prob:  3.53% Token: | 16|
Top 4th token. Logit: 15.35 Prob:  1.92% Token: | 17|
Top 5th token. Logit: 15.29 Prob:  1.82% Token: | 18|
Top 6th token. Logit: 15.25 Prob:  1.75% Token: | 12|
Top 7th token. Logit: 15.21 Prob:  1.68% Token: |
|
Top 8th token. Logit: 14.82 Prob:  1.14% Token: | 19|
Top 9th token. Logit: 13.83 Prob:  0.42% Token: | 20|


Ranks of the answer tokens: [(' 14', 0)]

In [ ]:
example_prompt = "2 4 6 8 10 12 14" # this is way too close. why does it get worse?
example_answer = " 16"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 6', ' 8', ' 10', ' 12', ' 14']
Tokenized answer: [' 16']


Performance on answer token:
Rank: 0        Logit: 18.04 Prob: 39.49% Token: | 16|

Top 0th token. Logit: 18.04 Prob: 39.49% Token: | 16|
Top 1th token. Logit: 18.01 Prob: 38.48% Token: | 15|
Top 2th token. Logit: 16.57 Prob:  9.10% Token: | 17|
Top 3th token. Logit: 15.65 Prob:  3.64% Token: | 18|
Top 4th token. Logit: 15.12 Prob:  2.15% Token: |
|
Top 5th token. Logit: 14.49 Prob:  1.14% Token: | 20|
Top 6th token. Logit: 13.78 Prob:  0.56% Token: | 14|
Top 7th token. Logit: 13.75 Prob:  0.54% Token: | 19|
Top 8th token. Logit: 12.81 Prob:  0.21% Token: | 21|
Top 9th token. Logit: 12.76 Prob:  0.20% Token: |15|


Ranks of the answer tokens: [(' 16', 0)]

In [ ]:
example_prompt = "2 4 6 8 10 12 14 16"
example_answer = " 18"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 6', ' 8', ' 10', ' 12', ' 14', ' 16']
Tokenized answer: [' 18']


Performance on answer token:
Rank: 0        Logit: 18.27 Prob: 54.41% Token: | 18|

Top 0th token. Logit: 18.27 Prob: 54.41% Token: | 18|
Top 1th token. Logit: 17.75 Prob: 32.11% Token: | 17|
Top 2th token. Logit: 15.38 Prob:  3.01% Token: | 19|
Top 3th token. Logit: 14.96 Prob:  1.97% Token: |
|
Top 4th token. Logit: 14.92 Prob:  1.90% Token: | 20|
Top 5th token. Logit: 14.21 Prob:  0.94% Token: | 16|
Top 6th token. Logit: 13.90 Prob:  0.69% Token: | 21|
Top 7th token. Logit: 13.32 Prob:  0.38% Token: | 22|
Top 8th token. Logit: 12.52 Prob:  0.17% Token: | 24|
Top 9th token. Logit: 12.29 Prob:  0.14% Token: | 23|


Ranks of the answer tokens: [(' 18', 0)]

In [ ]:
example_prompt = "2 4 6 8 10 12 14 18"
example_answer = " 20"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 6', ' 8', ' 10', ' 12', ' 14', ' 18']
Tokenized answer: [' 20']


Performance on answer token:
Rank: 0        Logit: 17.63 Prob: 36.05% Token: | 20|

Top 0th token. Logit: 17.63 Prob: 36.05% Token: | 20|
Top 1th token. Logit: 16.80 Prob: 15.70% Token: | 19|
Top 2th token. Logit: 16.54 Prob: 12.17% Token: | 22|
Top 3th token. Logit: 16.44 Prob: 11.02% Token: | 21|
Top 4th token. Logit: 15.81 Prob:  5.85% Token: | 24|
Top 5th token. Logit: 15.24 Prob:  3.33% Token: | 23|
Top 6th token. Logit: 15.14 Prob:  2.99% Token: | 18|
Top 7th token. Logit: 14.89 Prob:  2.33% Token: |
|
Top 8th token. Logit: 13.69 Prob:  0.70% Token: | 30|
Top 9th token. Logit: 13.59 Prob:  0.63% Token: | 25|


Ranks of the answer tokens: [(' 20', 0)]

## corrupt n-2

In [ ]:
example_prompt = "2 4 6 null 10 12"  #6 is minimal
example_answer = " 14"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 6', ' null', ' 10', ' 12']
Tokenized answer: [' 14']


Performance on answer token:
Rank: 10       Logit: 11.58 Prob:  2.62% Token: | 14|

Top 0th token. Logit: 12.62 Prob:  7.41% Token: | 0|
Top 1th token. Logit: 12.28 Prob:  5.26% Token: | 12|
Top 2th token. Logit: 12.28 Prob:  5.26% Token: | 13|
Top 3th token. Logit: 12.02 Prob:  4.06% Token: | null|
Top 4th token. Logit: 11.92 Prob:  3.65% Token: | 15|
Top 5th token. Logit: 11.77 Prob:  3.17% Token: | 1|
Top 6th token. Logit: 11.74 Prob:  3.06% Token: | 9|
Top 7th token. Logit: 11.73 Prob:  3.02% Token: | 11|
Top 8th token. Logit: 11.69 Prob:  2.93% Token: | 16|
Top 9th token. Logit: 11.60 Prob:  2.66% Token: |
|


Ranks of the answer tokens: [(' 14', 10)]

In [ ]:
example_prompt = "2 4 null 8 10 12"  #6 is minimal
example_answer = " 14"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' null', ' 8', ' 10', ' 12']
Tokenized answer: [' 14']


Performance on answer token:
Rank: 4        Logit: 13.35 Prob:  6.71% Token: | 14|

Top 0th token. Logit: 14.03 Prob: 13.22% Token: | 13|
Top 1th token. Logit: 13.61 Prob:  8.66% Token: | 12|
Top 2th token. Logit: 13.44 Prob:  7.33% Token: | 16|
Top 3th token. Logit: 13.37 Prob:  6.87% Token: | 15|
Top 4th token. Logit: 13.35 Prob:  6.71% Token: | 14|
Top 5th token. Logit: 12.60 Prob:  3.16% Token: | 0|
Top 6th token. Logit: 12.59 Prob:  3.14% Token: |
|
Top 7th token. Logit: 12.57 Prob:  3.08% Token: | 11|
Top 8th token. Logit: 12.47 Prob:  2.77% Token: | 17|
Top 9th token. Logit: 12.24 Prob:  2.21% Token: | 10|


Ranks of the answer tokens: [(' 14', 4)]